<h1><center> Email Automation - Canada Data</h1></center>

**Project Aim: Automated Email Response with LLM**

This project aims to automate email responses using Large Language Models (LLM). The focus is on enhancing communication efficiency through natural language understanding and generation. Key goals include efficient email handling, NLP integration, customization, adaptability, and scalability. The project utilizes advanced LLM, such as GPT-3, to comprehend diverse language structures and generate human-like responses, making it a user-friendly solution for streamlined email communication.

#### Read in the necessary libraries and packages

In [2]:
import os
import email
import copy
import pandas as pd
import extract_msg
import numpy as np
import re
import openai
import re, string
import emoji
import nltk
from sklearn import preprocessing
from cleantext import clean
from fuzzywuzzy import fuzz, process 
import time
import datetime
from bs4 import BeautifulSoup
import dask.dataframe as dd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import psycopg2
import pandas as pd
from sqlalchemy import create_engine

import boto3
import yaml
import base64
from datetime import datetime
from dateutil.relativedelta import relativedelta,FR,TU,WE

openai.api_key = "ENTER_YOUR_API_KEY" 

Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.
/home/ec2-user/.local/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


#### Function with OpenAI API Key

In [3]:
# imports
import random
import time

import openai

# define a retry decorator
def retry_with_exponential_backoff(
    func,
    initial_delay: float = 1,
    exponential_base: float = 2,
    jitter: bool = True,
    max_retries: int = 30,
    errors: tuple = (openai.error.RateLimitError,openai.error.APIError,openai.error.Timeout,openai.error.APIConnectionError,openai.error.ServiceUnavailableError),
):
    """Retry a function with exponential backoff."""

    def wrapper(*args, **kwargs):
        # Initialize variables
        num_retries = 0
        delay = initial_delay

        # Loop until a successful response or max_retries is hit or an exception is raised
        while True:
            try:
                return func(*args, **kwargs)

            # Retry on specific errors
            except errors as e:
                # Increment retries
                num_retries += 1

                # Check if max retries has been reached
                if num_retries > max_retries:
                    raise Exception(
                        f"Maximum number of retries ({max_retries}) exceeded."
                    )

                # Increment the delay
                delay *= exponential_base * (1 + jitter * random.random())

                # Sleep for the delay
                time.sleep(delay)

            # Raise exceptions for any errors not specified
            except Exception as e:
                raise e

    return wrapper

@retry_with_exponential_backoff
def get_completion(prompt, model="gpt-3.5-turbo"):
    """
    Generate completion using OpenAI ChatCompletion API.
    """
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message["content"]

#### Read in the Excel wih Expected Values

* expected and predicted suffix
* zipcode, city and country columns added.

In [ ]:
df = pd.read_excel('/home/ec2-user/python/Dhanya/email_automation/data/master_data_canada_18thAug_v3.xlsx')
df = df.rename(columns={c: 'expected_'+ c for c in df.columns if c not in ['body', 'emailid', 'Email', 'email_processed',
                                        'Exclusion Indicator','Comments_BG', 'Comments_BG_Binary']})

In [ ]:
df.head(2)

#### Remove Emails that arent exactly LCL Quotation requests

In [ ]:
df['Exclusion Indicator'].value_counts(normalize = True)

In [ ]:
df['Comments_BG_Binary'].value_counts(normalize = True)

In [ ]:
df.loc[(df['Comments_BG_Binary'] != df['Exclusion Indicator']), ['emailid','email_processed','Exclusion Indicator','Comments_BG_Binary','Comments_BG']].head()

**Preprocessing Functions to remove superflous information from the emails**

In [4]:
# this function is to remove the strings enclosed by either <> or []

def a(test_str):
    ret = ''
    skip1c = 0
    skip2c = 0
    for i in test_str:
        if i == '[':
            skip1c += 1
        elif i == '<':
            skip2c += 1
        elif i == ']' and skip1c > 0:
            skip1c -= 1
        elif i == '>'and skip2c > 0:
            skip2c -= 1
        elif skip1c == 0 and skip2c == 0:
            ret += i
    return ret

def clean_hashtags(tweet):
    """
    Remove hashtags from the text.
    """
    new_tweet = " ".join(word.strip() for word in re.split('#(?!(?:hashtag)\b)[\w-]+(?=(?:\s+#[\w-]+)*\s*$)', tweet))
    new_tweet2 = " ".join(word.strip() for word in re.split('#|_', new_tweet))
    return new_tweet2

def clean_regards(text):
    """
    Remove closing regards from the text.
    """
    text = re.split("best regards|regards|thanks & regards|thanks and regards|warm regards|kind regards|yours sincerely|sincerely|Atenciosamente, Regards, Saludos,", text)[0]
    return text

def clean_tail(text):
    """
    Clean unnecessary tail information from the text.
    """
    # text = re.split('our office working hours', text)[0]
    # text = re.split('important note:', text)[0]
    # text = re.split('note:', text)[0]
    text = re.split('phone:', text)[0]
    #text = re.split('from:', text)[0]
    text = re.split('mob no', text)[0]
    text = re.split('mob no:|mobile|email:', text)[0]
    
    #text = text.replace('caution: dear recipient, gulf craft will never communicate change of bank d= etails by email, if you receive any such emails from our pleas= e do not respond and contact our sales by telephone (preferable landline to= landline) and report the same to helpdesk in a separate e= -mail.','')
    #text = text.replace('caution: dear recipient, gulf craft will never communicate change of bank details by email, if you receive any such emails from our domain/user, please do not respond and contact our sales by telephone (preferable landline to landline) and report the same to helpdesk@gulfcraftinc.com in a separate e-mail.','')
    #text = text.replace('caution: dear recipient, gulf craft will never communicate change of bank details by email, if you receive any such emails from our domain/user, please do not respond and contact our sales by telephone (preferable landline to landline) and report the same to helpdesk in a separate e-mail.','')
    #text = text.replace('*we strictly dont entertain quotation which includes iranian vessel, routing via iran or iran flagged vessel','')
    text = text.replace('**this mail is from outside our organization. treat hyperlinks and attachments in this mail with caution**','')
    return text

def strip_all_entities(text):
    """
    Remove punctuations, links, mentions, and new line characters from the text.
    """
    text = text.replace('_x000D_','').replace('\r', '').replace('\n', ' ').replace('\n', ' ').lower()
    text = re.sub(r"(?:\@|https?\://)\S+", "", text)
    text = re.sub(r'[^\x00-\x7f]',r'', text)
    banned_list = string.punctuation + 'Ã'+'±'+'ã'+'¼'+'â'+'»'+'§'
    text = " ".join(word.strip() for word in re.split('#|_', text))
    return text

def remove_mult_spaces(text):
    """
    Remove multiple spaces from the text.
    """
    return re.sub("\s\s+" , " ", text)

def filter_chars(text):
    """
    Filter special characters such as & and $ present in some words.
    """
    sent = []
    for word in a.split(' '):
        if ('$' in word) or ('&' in word) or ('/' in word):
            sent.append('')
        else:
            sent.append(word)
    return ' '.join(sent)

def preprocess_text(text):
    """
    Preprocess the text by applying various cleaning functions.
    """
    
    text = a(text)
    text = strip_all_entities(text)
    text = remove_mult_spaces(text)
    # text = clean_regards(text)
    text = clean_tail(text)
    text = clean_hashtags(text)
    return text

Applying the preprocessing function on our emails.

In [ ]:
df['email_processed'] = df['email_processed'].apply(preprocess_text)

## Classification function to determine the type of Request

In [ ]:
def make_classification(email):
    prompt = f"""
    The task is to categorize an email into one of four classes: "Quotation Request", "Track and Trace Request"
    , "Sailing Schedule Request" and "Random Conversation".
    A "Quotation Request" is a customer's request for pricing information, including origin and destination locations, and cargo details.
    If the email is not a "Quotation Request", check for the next three classes.
    A "Sailing Schedule Request" is a customer request for sailing schedule or vessel schedule indicated by keywords like "sailing schedule request", "vessel availability", "next vessel" or "vessel schedule".
    A "Track and Trace Request" asks about the status, ETA or delivery status of a shipment. Look for key phrases like "update us on the subject shipment", "status of cargo", "delivery status" or "when will it be delivered?".
    Any email which looks like a reply to a "Quotation Request" is a "Random Conversation".
    Any email which looks like a reply to a "Sailing Schedule Request" is a "Random Conversation".
    Any email which looks like a reply to a "Track and Trace Request" is a "Random Conversation".
    Any email which does not fall under the above 3 classes is a "Random Conversation".
    Give the result as only the class name and nothing else.
    Review: ```{email}```
    """
    response = get_completion(prompt)
    return response

In [ ]:
classify = ['predicted_class']
df[classify] = ''
for i, value in enumerate(df['email_processed']):
    result = make_classification(value)
    df.loc[i, classify] = result
    del(value)

In [ ]:
df['predicted_class'].value_counts() #.index

#### Extracting the Quotation Requests

In [ ]:
df_quotation = df.loc[df['predicted_class'] == 'Quotation Request'].reset_index(drop=True)

#### New Addition - Remove Email Signature / Footer 

In [ ]:
def extract_information_from(email):
    prompt = f"""
    Remove the email signature block.

    Remove any disclaimers found as well.
      
    Review: ```{email}```
    
    """
    response = get_completion(prompt)
    return response


# Subject line has to be retained

In [ ]:
for i, value in enumerate(df_quotation['email_processed']):
    result = extract_information_from(value)
    df_quotation.loc[i, 'email_processed_revised'] = result
    del(value)

In [ ]:
df_quotation['email_processed_revised'] = df_quotation['email_processed_revised'].str.replace('Review: ```',' ')
df_quotation['email_processed_revised'] = df_quotation['email_processed_revised'].str.replace('Review:', ' ')
df_quotation['email_processed_revised'] = df_quotation['email_processed_revised'].str.replace(' tml ', 'terminal')
df_quotation['email_processed'] = df_quotation['email_processed_revised'].str.replace('^\W+','' )
df_quotation['email_processed'].head()

<h1><center> Read in the Cleaned and Processed Emails </h1></center>

#### Try RegEx for Origin/Destination Type Classification

In [ ]:
# df_quotation.loc[df_quotation['email_processed'].str.contains('shipment from turkey to toronto'),['email_processed'] ]

# df_quotation[df_quotation["email_processed"].str.contains(r'\b(shipment from) (?s).*? (to).+\b', regex=True, case=False)] 

# df_quotation[df_quotation["email_processed"].str.contains(r'\b(cfs) (?s).*? (to door).+\b', regex=True, case=False)] 

# df_quotation[df_quotation["email_processed"].str.contains(r'\b(pol:) (?s).*? (pod:).+\b', regex=True, case=False)] 

#### Read in the CFS Destination File - It gives us the Port Codes

In [ ]:
last_fri_dt1 = (datetime.now() + relativedelta(weekday=FR(-1))).strftime('%Y-%m-%d')
mapping_data = dd.read_parquet('s3://ds-data-model/ECU_MASTER/CFSDestination/dt='+ last_fri_dt1,engine="fastparquet").compute()
mapping_data.rename(columns={'Country_Id_Name' : 'Country'}, inplace=True)
mapping_data.drop_duplicates(inplace = True)
mapping_data = mapping_data.loc[:,['Iata', 'Port','UnCode','Country']].apply(lambda x : x.str.strip().str.upper()) 
# Remove airport codes

keywords_to_remove = ['APT', 'AIRPORT']
#ports_to_remove = ['CANWP', 'CAVST']
mapping_data.dropna(inplace=True)#.reset_index(drop=True)
mapping_data.drop_duplicates(inplace=True)#.reset_index(drop=True)
mapping_data = mapping_data.loc[~(mapping_data['Port'].str.contains('|'.join(keywords_to_remove), case=False)==True)]

#mapping_data = mapping_data.loc[~(mapping_data['UnCode'].str.contains('|'.join(ports_to_remove), case=False, na=False)==True)]
# Remove Ports that ends in .

mapping_data = mapping_data[~mapping_data['Port'].str.endswith('.')]
mapping_data.head()

In [ ]:
mapping_data.shape

#### Function to extract Origin and Destination from email.

In [ ]:
#     If the keywords like ('cfs', 'terminal', 'freezone','tml','term') are associated with origin or destination. Extract those along with these values.
#     Keywords like ('dap','ddp') indicate delivery location.

In [ ]:
def extract_information_from(email):
    prompt = f"""

    Please provide origin location (full address) and destination location (full address) in the following format:
    
    Origin Location:
    Destination Location:
    
    The response should include only 2 lines with the field mentioned above.
    Don't include anything other than these 2 lines and 2 fields even if the field is missing and there is insufficient information.
    
    Review: ```{email}```
    
    """
    response = get_completion(prompt)
    return response

In [ ]:
for i, value in enumerate(df_quotation['email_processed']):
    result = extract_information_from(value)
    df_quotation.loc[i, 'predicted_origin_destination'] = result
    del(value)

In [ ]:
df_quotation.head(2)

In [ ]:
# cleaning the extracted origin and destination column returned by ChatGPT

df_quotation['predicted_origin_destination'] = df_quotation['predicted_origin_destination'].str.replace('Origin Location:\n', '')

In [ ]:
# splitting the predicted origin and destination into two separate strings

for i, value in enumerate(df_quotation['email_processed']):
    input_string = df_quotation.loc[i, 'predicted_origin_destination']
    split_parts = input_string.split('\n', 1)
    try:
        if(len(split_parts)==2):
            df_quotation.loc[i, 'predicted_origin'] = split_parts[0]
            df_quotation.loc[i, 'predicted_destination'] = split_parts[1]
        elif(len(split_parts==1)):
            df_quotation.loc[i, 'predicted_destination'] = split_parts[0]
    except:
        df_quotation.loc[i, 'predicted_origin'] = ''
        df_quotation.loc[i, 'predicted_destination'] = ''

In [ ]:
df_quotation_input = df_quotation.copy()

# cleaning the extracted origin and destination

df_quotation_input['predicted_origin'] = df_quotation_input['predicted_origin'].str.replace('Origin Location: ', '')
df_quotation_input['predicted_origin'] = df_quotation_input['predicted_origin'].str.replace('Origin Location:', '')

df_quotation_input['predicted_destination'] = df_quotation_input['predicted_destination'].str.replace('\nDestination Location:','')
df_quotation_input['predicted_destination'] = df_quotation_input['predicted_destination'].str.replace('\nDestination Location: ','')
df_quotation_input['predicted_destination'] = df_quotation_input['predicted_destination'].str.replace('Destination Location:','')
df_quotation_input['predicted_destination'] = df_quotation_input['predicted_destination'].str.replace('\n','')

In [ ]:
df_quotation_input.shape[0]

In [ ]:
print('No. of entries where Origin entries are null :',df_quotation_input[df_quotation_input['predicted_origin']==''].shape[0])
print('No. of entries where Destination entries are null : ',df_quotation_input[df_quotation_input['predicted_destination']==''].shape[0])

#### Analyze the entries that ChatGPT has failed to predict the Origin and Destination 

In [ ]:
# df_quotation_input.loc[(df_quotation_input['predicted_origin']=='') | (df_quotation_input['predicted_destination']==''),
#                       ['emailid','email_processed','predicted_origin','predicted_destination']].to_excel('Null_Prediction.xlsx')

In [ ]:
df_quotation_input.loc[(df_quotation_input['predicted_origin'] == '') | (df_quotation_input['predicted_destination'] == ''),
                      ['emailid','email_processed','predicted_origin','predicted_destination']]

**Removing instances where expected value is null**

In [ ]:
df_quotation_input = df_quotation_input[~(df_quotation_input['expected_Origin_type'].isnull())  &
                  ~(df_quotation_input['expected_Destination_type'].isnull())]

df_quotation_input.shape

In [ ]:
df_quotation_input = df_quotation_input.loc[~(df_quotation_input['expected_emailfrom'].isnull())]
df_quotation_input = df_quotation_input.loc[~(df_quotation_input['expected_emailto'].isnull())]
df_quotation_input.shape

#### Exlude irrelevant emails

In [ ]:
df_quotation_input = df_quotation_input[(df_quotation_input['Comments_BG_Binary'] !='EXCLUDE')]
df_quotation_input.shape

<h1><center> Metric 1 : Derive the OriginType and Destintion Type </h1></center>

In [ ]:
df_quotation_input['predicted_origin'] = df_quotation_input['predicted_origin'].str.lower()
df_quotation_input['predicted_destination'] = df_quotation_input['predicted_destination'].str.lower()
df_quotation_input['email_processed'] = df_quotation_input['email_processed'].str.lower()

#### Step 1 : Hardcoded rules to classify port and door from the unparsed origin and destination.

In [ ]:
origin_door_keywords = ['door to door','door to port', 'door to cfs' , 'door to terminal', 'door to term',
                        'door - door', 'door - port', 'door : door', 'door : port', 
                        'door-door', 'door-port', 'from door', 'pickup location',
                        'from location',' pick up ', ' pickup ', 'pick-up', 'pickup address', 'pick up address', 
                       'pu charge', 'p/u', ' door pickup ', ' from location ',
                       ' exw ', 'exworks','ex works', 'ex-works', 'ex - works', 'ex work','exwork',
                        'supplier address','door/door']

destination_door_keywords = ['port to door','door to door', 'cfs to door', 'terminal to door',
                        'port - door', 'door - door', 'port-door', 'door-door','port : door','door : door',
                        'shipment to door', ' dap ', 'delivery address', 'door delivery', 'delivery at place',
                ' ddp ','ddp ' ' place of reciept ','destination delivery','final destination',
            ' to location ','delivery to door', 'residential delivery', 'rate to door', 'delivery at door','door/door']

In [ ]:
df_quotation_input['predicted_fromtype'] = np.where( df_quotation_input['email_processed'].str.contains('|'.join(origin_door_keywords)), 'door', 'port')
df_quotation_input['predicted_totype'] = np.where( df_quotation_input['email_processed'].str.contains('|'.join(destination_door_keywords)), 'door', 'port')

#### Update the Origin/ Destination classification Method indicator

In [ ]:
print(df_quotation_input['predicted_fromtype'].unique())
print(df_quotation_input['predicted_totype'].unique())

In [ ]:
# Origin Type Match Mismatch

df_quotation_input.loc[df_quotation_input['predicted_fromtype'] != df_quotation_input['expected_Origin_type'], ['emailid','email_processed',
                                            'predicted_origin','predicted_fromtype','expected_Origin_type']]

In [ ]:
# Destination Type Match Mismatch

df_quotation_input.loc[df_quotation_input['predicted_totype'] != df_quotation_input['expected_Destination_type'], ['emailid','email_processed',
                                            'predicted_destination','predicted_totype','expected_Destination_type']]

- Both scenarios are misclassified due to ambiguity in the emails

#### Intermediate Check for Origin/Destination Type match

In [ ]:
# Origin Type Match 

df_quotation_input[df_quotation_input['predicted_fromtype'] == df_quotation_input['expected_Origin_type']].shape[0]/df_quotation_input.shape[0]

In [ ]:
# Destination Type Match

df_quotation_input[df_quotation_input['predicted_totype'] == df_quotation_input['expected_Destination_type']].shape[0]/df_quotation_input.shape[0]

#### Remove instances where the Expected Values are null (else it will distort the accuracy)

- Origin / Destination Type is considered for now
- We will have to add/ switch to other columns too as we progress testing metric by metric

In [ ]:
df_quotation_input[~(df_quotation_input['expected_Origin_type'].isnull())  &
                  ~(df_quotation_input['expected_Destination_type'].isnull())].shape

In [ ]:
df_quotation_input = df_quotation_input[~(df_quotation_input['expected_Origin_type'].isnull())  &
                  ~(df_quotation_input['expected_Destination_type'].isnull())]

df_quotation_input.shape

In [ ]:
df_quotation_input.loc[df_quotation_input['expected_emailfrom'].isnull()]

In [ ]:
df_quotation_input = df_quotation_input.loc[~(df_quotation_input['expected_emailfrom'].isnull())]
df_quotation_input = df_quotation_input.loc[~(df_quotation_input['expected_emailto'].isnull())]
df_quotation_input.shape

#### Add an indicator to identify if the Predicted and Expected values of Origin/Destination Type match or not

- We have to add this for other columns too as we progress

In [ ]:
df_quotation_input['OriginType_Match'] = np.where((df_quotation_input['predicted_fromtype'] == df_quotation_input['expected_Origin_type']),
           'Matched', 'Not Matched' )

df_quotation_input['DestinationType_Match'] = np.where((df_quotation_input['predicted_totype'] == df_quotation_input['expected_Destination_type']),
           'Matched', 'Not Matched' )

In [ ]:
df_quotation_input['OriginType_Match'].value_counts(normalize= True)

In [ ]:
df_quotation_input['DestinationType_Match'].value_counts(normalize= True)

<h1><center> Analyze the extracted origin and destination columns </h1></center>

In [ ]:
df_quotation_input.loc[df_quotation_input['email_processed'].str.contains('henri'),['email_processed',
                                    'predicted_fromtype','expected_Origin_type',
                                    'predicted_totype','expected_Destination_type',
                                    'predicted_origin','predicted_destination']]

<h1><center> Measuring Accuracy for Origin / Destination Type </h1></center>

In [ ]:
# Auto encode the columns to be measured 

df_quotation_input['expected_Origin_type'] = np.where(df_quotation_input['expected_Origin_type']=='port', 1, 0)
df_quotation_input['predicted_fromtype'] = np.where(df_quotation_input['predicted_fromtype']=='port', 1, 0)

In [ ]:
def comp_confmat(actual, predicted):
    # extract the different classes
    classes = np.unique(actual)
    # initialize the confusion matrix
    confmat = np.zeros((len(classes), len(classes)))
    # loop across the different combinations of actual / predicted classes
    for i in range(len(classes)):
        for j in range(len(classes)):
            # count the number of instances in each combination of actual / predicted
            confmat[i, j] = np.sum((actual==classes[i]) & (predicted == classes[j]))
    return confmat

print(comp_confmat(df_quotation_input['expected_Origin_type'], df_quotation_input['predicted_fromtype']))

In [ ]:
OriginType_Confusion_Matrix = pd.crosstab(df_quotation_input['predicted_fromtype'], df_quotation_input['expected_Origin_type'])
OriginType_Confusion_Matrix.columns = ['Expected - Door', 'Expected - Port']
OriginType_Confusion_Matrix = OriginType_Confusion_Matrix.reset_index()
OriginType_Confusion_Matrix['predicted_fromtype'] = np.where((OriginType_Confusion_Matrix['predicted_fromtype'] == 0), 'Predicted - Door', 'Predicted - Port')
OriginType_Confusion_Matrix.rename(columns = {'predicted_fromtype' : 'OriginType'}, inplace = True)
OriginType_Confusion_Matrix

#### Summary of Origin Type Prediction

In [ ]:
# Accuracy
OriginType_Accuracy = ((OriginType_Confusion_Matrix['Expected - Door'][0] + OriginType_Confusion_Matrix['Expected - Port'][1])/(OriginType_Confusion_Matrix['Expected - Door'][0] + OriginType_Confusion_Matrix['Expected - Port'][0] + 
OriginType_Confusion_Matrix['Expected - Door'][1] + OriginType_Confusion_Matrix['Expected - Port'][1]))*100
print('Accuracy of Origin Type : ' , OriginType_Accuracy)

# Recall of Door Class within OriginType
OriginType_DoorRecall = ((OriginType_Confusion_Matrix['Expected - Door'][0])/(OriginType_Confusion_Matrix['Expected - Door'][0] + 
OriginType_Confusion_Matrix['Expected - Door'][1] ))*100
print('Recall of Door class with Origin Type : ' , OriginType_DoorRecall)

# Precision of Door Class within OriginType
OriginType_DoorPrecision = ((OriginType_Confusion_Matrix['Expected - Door'][0])/(OriginType_Confusion_Matrix['Expected - Door'][0] + 
OriginType_Confusion_Matrix['Expected - Port'][0] ))*100
print('Precision of Door class with Origin Type : ' , OriginType_DoorPrecision)

# Recall of Port Class within OriginType
OriginType_PortRecall = ((OriginType_Confusion_Matrix['Expected - Port'][1])/(OriginType_Confusion_Matrix['Expected - Port'][0] + 
OriginType_Confusion_Matrix['Expected - Port'][1] ))*100
print('Recall of Port class with Origin Type : ' , OriginType_PortRecall)

# Precision of Port Class within OriginType
OriginType_PortPrecision = ((OriginType_Confusion_Matrix['Expected - Port'][1])/(OriginType_Confusion_Matrix['Expected - Port'][1] + 
OriginType_Confusion_Matrix['Expected - Door'][1] ))*100
print('Precision of Port class with Origin Type : ' , OriginType_PortPrecision)

# Orgin Type correct Match % using Email Keywords 
#origintype_emailmethod = df_quotation_input[(df_quotation_input['predicted_fromtype_from'] == 'email_keywords') & (df_quotation_input['OriginType_Match'] == 'Matched')].shape[0]/df_quotation_input[df_quotation_input['predicted_fromtype_from'] == 'email_keywords'].shape[0]
#print('% of OriginType entries correctly matched using Email Keywords : ' , origintype_emailmethod*100)

# Orgin Type correct Match % using ChatGPT 
#origintype_chatgptmethod = df_quotation_input[(df_quotation_input['predicted_fromtype_from'] != 'email_keywords') & (df_quotation_input['OriginType_Match'] == 'Matched')].shape[0]/df_quotation_input[df_quotation_input['predicted_fromtype_from'] != 'email_keywords'].shape[0]
#print('% of OriginType entries correctly matched using ChatGPT : ' , origintype_chatgptmethod*100)

#### Analyze the mismatched Origin Types

In [ ]:
df_quotation_input.loc[(df_quotation_input['OriginType_Match'] != 'Matched'), ['emailid','email_processed',
                                            'predicted_origin','predicted_fromtype','expected_Origin_type']].head()

In [ ]:
df_quotation_input.loc[
                       (df_quotation_input['email_processed'].str.contains('henri')),['emailid','email_processed','expected_Origin_type','expected_Destination_type']]

#### Summary Destination Type Prediction 

In [ ]:
# Auto encode the Expected and Predicted OriginType Columns 

df_quotation_input['expected_Destination_type'] = np.where(df_quotation_input['expected_Destination_type']=='port', 1, 0)

df_quotation_input['predicted_totype'] = np.where(df_quotation_input['predicted_totype']=='port', 1, 0)

DestinationType_Confusion_Matrix = pd.crosstab(df_quotation_input['predicted_totype'], df_quotation_input['expected_Destination_type'])
DestinationType_Confusion_Matrix.columns = ['Expected - Door', 'Expected - Port']
DestinationType_Confusion_Matrix = DestinationType_Confusion_Matrix.reset_index()
DestinationType_Confusion_Matrix['predicted_totype'] = np.where((DestinationType_Confusion_Matrix['predicted_totype'] == 0), 'Predicted - Door', 'Predicted - Port')
DestinationType_Confusion_Matrix.rename(columns = {'predicted_totype' : 'DestinationType'}, inplace = True)
DestinationType_Confusion_Matrix

In [ ]:
# Accuracy
DestinationType_Accuracy = ((DestinationType_Confusion_Matrix['Expected - Door'][0] + DestinationType_Confusion_Matrix['Expected - Port'][1])/(DestinationType_Confusion_Matrix['Expected - Door'][0] + DestinationType_Confusion_Matrix['Expected - Port'][0] + 
DestinationType_Confusion_Matrix['Expected - Door'][1] + DestinationType_Confusion_Matrix['Expected - Port'][1]))*100
print('Accuracy of Destination Type : ' , DestinationType_Accuracy)

# Recall of Door Class within DestinationType
DestinationType_DoorRecall = ((DestinationType_Confusion_Matrix['Expected - Door'][0])/(DestinationType_Confusion_Matrix['Expected - Door'][0] + 
DestinationType_Confusion_Matrix['Expected - Door'][1] ))*100
print('Recall of Door class with Destination Type : ' , DestinationType_DoorRecall)

# Precision of Door Class within DestinationType
DestinationType_DoorPrecision = ((DestinationType_Confusion_Matrix['Expected - Door'][0])/(DestinationType_Confusion_Matrix['Expected - Door'][0] + 
DestinationType_Confusion_Matrix['Expected - Port'][0] ))*100
print('Precision of Door class with Destination Type : ' , DestinationType_DoorPrecision)

# Recall of Port Class within DestinationType
DestinationType_PortRecall = ((DestinationType_Confusion_Matrix['Expected - Port'][1])/(DestinationType_Confusion_Matrix['Expected - Port'][0] + 
DestinationType_Confusion_Matrix['Expected - Port'][1] ))*100
print('Recall of Port class with Destination Type : ' , DestinationType_PortRecall)

# Precision of Port Class within DestinationType
DestinationType_PortPrecision = ((DestinationType_Confusion_Matrix['Expected - Port'][1])/(DestinationType_Confusion_Matrix['Expected - Port'][1] + 
DestinationType_Confusion_Matrix['Expected - Door'][1] ))*100
print('Precision of Port class with Destination Type : ' , DestinationType_PortPrecision)

# Destination Type correct Match % using Email Keywords 
# destinationtype_emailmethod = df_quotation_input[(df_quotation_input['predicted_totype_from'] == 'email_keywords') & (df_quotation_input['DestinationType_Match'] == 'Matched')].shape[0]/df_quotation_input[df_quotation_input['predicted_totype_from'] == 'email_keywords'].shape[0]
# print('% of DestinationType entries correctly matched using Email Keywords : ' , destinationtype_emailmethod*100)

# # Destination Type correct Match % using ChatGPT 
# destinationtype_chatgptmethod = df_quotation_input[(df_quotation_input['predicted_totype_from'] != 'email_keywords') & (df_quotation_input['DestinationType_Match'] == 'Matched')].shape[0]/df_quotation_input[df_quotation_input['predicted_totype_from'] != 'email_keywords'].shape[0]
# print('% of DestinationType entries correctly matched using ChatGPT : ' , destinationtype_chatgptmethod*100)

#### Analyze the mismatched Destintation Types

In [ ]:
df_quotation_input.loc[(df_quotation_input['DestinationType_Match'] != 'Matched'), ['emailid','email_processed','predicted_destination','predicted_totype','expected_Destination_type']].head()

In [ ]:
# temp  = df_quotation_input[['email_processed','expected_Origin_type', 'expected_Destination_type',
#                             'predicted_fromtype', 'predicted_totype',
#        'predicted_fromtype_from', 'predicted_totype_from', 'OriginType_Match',
#        'DestinationType_Match']]
# temp.head()

In [ ]:
# df_quotation_input[['predicted_origin', 'expected_emailfrom', 
#                     'predicted_destination', 'expected_emailto']]

<h1><center> Clean the Predicted Origin and Destination Columns </h1></center>

- These extracted columns contain superflous words/sentences that bring down the accuracy

- Remove Keywords like :  'dap charges', 'door', 'not provided', 'not provided.', 'unknown' etc

#### Removing these hardcoded techniques

In [ ]:
# df_quotation_input[['predicted_origin', 'predicted_destination']]

In [ ]:
# # Remove Keywords from Origin

# df_quotation_input['predicted_origin'] = df_quotation_input['predicted_origin'].str.replace('dap charges','')
# df_quotation_input['predicted_origin'] = df_quotation_input['predicted_origin'].str.replace('door','')
# df_quotation_input['predicted_origin'] = df_quotation_input['predicted_origin'].str.replace('not provided.','')
# df_quotation_input['predicted_origin'] = df_quotation_input['predicted_origin'].str.replace('not provided','')
# df_quotation_input['predicted_origin'] = df_quotation_input['predicted_origin'].str.replace('unknown','')
# df_quotation_input['predicted_origin'] = df_quotation_input['predicted_origin'].str.replace('please advise address','')
# df_quotation_input['predicted_origin'] = df_quotation_input['predicted_origin'].str.replace('(please advise address)','')

# # Remove keywords from Destination

# df_quotation_input['predicted_destination'] = df_quotation_input['predicted_destination'].str.replace('dap charges','')
# df_quotation_input['predicted_destination'] = df_quotation_input['predicted_destination'].str.replace('door','')
# df_quotation_input['predicted_destination'] = df_quotation_input['predicted_destination'].str.replace('not provided.','')
# df_quotation_input['predicted_destination'] = df_quotation_input['predicted_destination'].str.replace('not provided','')
# df_quotation_input['predicted_destination'] = df_quotation_input['predicted_destination'].str.replace('unknown','')
# df_quotation_input['predicted_destination'] = df_quotation_input['predicted_destination'].str.replace('please advise address','')
# df_quotation_input['predicted_destination'] = df_quotation_input['predicted_destination'].str.replace('(please advise address)','')

###### Cleaning Steps :

* **For Port**
1. Get Uncode for port (both origin and destination)
2. Clean the origin and destination 
3. fuzzy match with cfs destination file.
4. look through values to decide score criteria
5. if fuzzy match fails, then look for python library which can give us uncode
6. then chatgpt prompt
7. If the uncode is null after these steps, then classify it as door and do the preprocessing of door

* **For Door**
1. Use chatgpt prompt to extract city, zipcode and country
2. If zipcode is missing, take help of library to get unique zipcode
3. If the location doesn't have unique zipcode, classify it as port -> then fuzzy match for port and then chagpt
4. if the location has city, zipcode and country. then do post processing for required output

* Finally calculate the accuracy

In [ ]:
mapping_data.head()

In [ ]:
mapping_data.shape

<h1><center> Metric 2 : Port/ Location Extraction and Cleaning - Origin + Destination </h1></center>

In [ ]:
# mapping_data['Country'] = mapping_data['Country'].str.replace('TAIWAN, CHINA', 'TAIWAN')
country_list = mapping_data['Country'].str.lower().to_list()

In [ ]:
# df['predicted_fromtype'] = np.where(df['predicted_fromtype']=='1', 'port', 'door')
# df['predicted_totype'] = np.where(df['predicted_totype']=='1', 'port', 'door')

#### Step 1 : Fuzzy Match with the CFS Mapping File to derive the Port UNCODES

In [ ]:
from fuzzywuzzy import fuzz, process

def fuzzy_match(name, choices):
    return process.extractOne(name, choices, scorer=fuzz.token_set_ratio)

def to_string(x):
    try:
        return str(x)
    except:
        return ''

# 'predicted_origin', 'predicted_destination'

def port_codes_mapping(df1):
    df = df1.copy()
    df['predicted_origin'] = df['predicted_origin'].fillna('')
    df['predicted_destination'] = df['predicted_destination'].fillna('')
    # stripping the values in order to remove empty space prsent before and after the string
    df['predicted_origin'] = df['predicted_origin'].str.strip()
    df['predicted_destination'] = df['predicted_destination'].str.strip()
    df['predicted_origin'] = df['predicted_origin'].replace(np.nan, '')
    df['predicted_destination'] = df['predicted_destination'].replace(np.nan, '')
    df['predicted_origin_match'] = df['predicted_origin'].apply(fuzzy_match, args=(mapping_data['Port'],))
    df[['predicted_origin_match_port', 'predicted_origin_match_score', 
        'predicted_origin_match_junk']] = pd.DataFrame(df['predicted_origin_match'].tolist(), index=df.index)
    df = df.applymap(to_string)
    df['predicted_destination_match'] = df['predicted_destination'].apply(fuzzy_match, args=(mapping_data['Port'],))
    df[['predicted_destination_match_port', 'predicted_destination_match_score',
       'predicted_destination_match_junk']] = pd.DataFrame(df['predicted_destination_match'].tolist(), index=df.index)
    quotations_origin_match = pd.merge(df, mapping_data[['Port','UnCode']],how='left',
                                      left_on='predicted_origin_match_port',
                                      right_on='Port')
    quotations_origin_match = quotations_origin_match.rename(columns={
        'UnCode':'predicted_origin_uncode'})
    quotations_destination_match = pd.merge(quotations_origin_match,
                            mapping_data[['Port','UnCode']],how='left',
                                left_on='predicted_destination_match_port',
                                            right_on='Port')
    quotations_destination_match = quotations_destination_match.rename(columns={
        'UnCode':'predicted_destination_uncode'})
    quotations_destination_match['predicted_origin_match_score'] = quotations_destination_match['predicted_origin_match_score'].astype(int)
    quotations_destination_match['predicted_destination_match_score'] = quotations_destination_match['predicted_destination_match_score'].astype(int)
    quotations_destination_match.loc[(quotations_destination_match['predicted_origin_match_score']>=65) & (~quotations_destination_match['predicted_origin'].isin(['not provided', 'unknown'])) & (~quotations_destination_match['predicted_origin'].isin(country_list)), 'predicted_origin_port'] = quotations_destination_match['predicted_origin_uncode']
    quotations_destination_match.loc[(quotations_destination_match['predicted_destination_match_score']>=65) & (~quotations_destination_match['predicted_destination'].isin(['not provided', 'unknown'])) & (~quotations_destination_match['predicted_destination'].isin(country_list)), 'predicted_destination_port'] = quotations_destination_match['predicted_destination_uncode']
    return quotations_destination_match

In [ ]:
# df['predicted_destination_match_score'].value_counts().index

In [ ]:
# df['predicted_origin_match_score'].astype(int).value_counts().index

In [ ]:
df = port_codes_mapping(df_quotation_input)

In [ ]:
df[['predicted_origin','predicted_fromtype', 'predicted_origin_port', 'predicted_origin_match_score', 
        'predicted_origin_match_junk', 'predicted_destination','predicted_totype','predicted_destination_port', 'predicted_destination_match_score',
       'predicted_destination_match_junk']].head()

In [ ]:
print(df['predicted_fromtype'].unique())
print(df['predicted_totype'].unique())

In [ ]:
# Change Auto encoded columns

df['predicted_fromtype'] = np.where((df['predicted_fromtype'] == '1'), 'port', 'door')
df['predicted_totype'] = np.where((df['predicted_totype'] == '1'), 'port', 'door')

In [ ]:
print(df['predicted_fromtype'].unique())
print(df['predicted_totype'].unique())

In [ ]:
df[['expected_emailfrom','expected_emailto' ]] = df[['expected_emailfrom','expected_emailto' ]].apply(lambda x : x.str.strip().str.upper())

In [ ]:
# Analyze the Origin Ports match score

print('% of Origin Ports that have been mapped to a UNCODE with a fuzzy score of 100: ',(df[(df['predicted_fromtype'] =='port') & (df['predicted_origin_match_score'] == 100)].shape[0]/df[(df['predicted_fromtype'] =='port')].shape[0])*100)
# Analyze the Origin Ports match score

print('% of Destination Ports that have been mapped to a UNCODE with a fuzzy score of 100: ',(df[(df['predicted_totype'] =='port') & (df['predicted_destination_match_score'] == 100)].shape[0]/df[(df['predicted_totype'] =='port')].shape[0])*100)

In [ ]:
df.loc[(df['predicted_fromtype'] == 'port') & (df['predicted_origin_port'] != df['expected_emailfrom']),['emailid','email_processed','predicted_fromtype','expected_Origin_type','predicted_origin',
         'predicted_origin_uncode','predicted_origin_match_score','predicted_origin_port',
                                              'expected_emailfrom']]

In [ ]:
# # Analyze the Origin Ports with a fuzzy score < 100

# df.loc[(df['predicted_origin_match_score'] < 100) & (df['predicted_fromtype'] =='port'),['predicted_origin', 'predicted_origin_port', 'predicted_origin_match_score', 
#         'predicted_origin_match_junk', 'predicted_destination','predicted_destination_port', 'predicted_destination_match_score',
#        'predicted_destination_match_junk']].sort_values(by = 'predicted_origin_match_score', ascending = True)

In [ ]:
# # Analyze the Destination Ports with a fuzzy score < 100

# df.loc[(df['predicted_destination_match_score'] < 100) & (df['predicted_totype'] =='port'),['predicted_origin', 'predicted_origin_port', 'predicted_origin_match_score', 
#         'predicted_origin_match_junk', 'predicted_destination','predicted_destination_port', 'predicted_destination_match_score',
#        'predicted_destination_match_junk']].sort_values(by = 'predicted_destination_match_score', ascending = True)

#### Step 2 : If any Origin/Destination Ports have no UNCODES found - Try looking up using ChatGPT

In [ ]:
df['predicted_destination_port'].value_counts().index

In [ ]:
def port_code(location):
    prompt = f"""
    What is the UN/LOCCODE for {location} sea port? Result should be only the code and nothing else.
    """
    response = get_completion(prompt)
    return response

In [ ]:
df['predicted_origin_port'] = df['predicted_origin_port'].replace(np.nan, '')
df['predicted_destination_port'] = df['predicted_destination_port'].replace(np.nan, '')

In [ ]:
df[['predicted_fromtype','predicted_origin_port','predicted_origin','predicted_totype','predicted_destination','predicted_destination_port']].head()

In [ ]:

# predicted_destination
# predicted_destination_match_score


In [ ]:

for i, value in enumerate(df['email_processed']):
    fromtype = df.loc[i, 'predicted_fromtype']
    totype = df.loc[i, 'predicted_totype']
    predicted_origin_port = df.loc[i, 'predicted_origin_port']
    predicted_destination_port = df.loc[i, 'predicted_destination_port']
    predicted_origin = df.loc[i, 'predicted_origin']
    predicted_destination = df.loc[i, 'predicted_destination']
    
    if(('port' in fromtype.lower())&(predicted_origin_port!='')):
        df.loc[i, 'predicted_emailfrom'] = predicted_origin_port

    if(('port' in fromtype.lower())&(predicted_origin_port=='')):
        df.loc[i, 'predicted_emailfrom'] = port_code(predicted_origin)
        
    if(('port' in totype.lower())&(predicted_destination_port!='')):
        df.loc[i, 'predicted_emailto'] = predicted_destination_port
        
    if(('port' in totype.lower())&(predicted_destination_port=='')):
        df.loc[i, 'predicted_emailto'] = port_code(predicted_destination)
        

In [ ]:
df['predicted_emailfrom'] = df['predicted_emailfrom'].str.replace(' ', '')
df['predicted_emailto'] = df['predicted_emailto'].str.replace(' ', '')
df['predicted_emailfrom'] = df['predicted_emailfrom'].astype(str)
df['predicted_emailto'] = df['predicted_emailto'].astype(str)
df['predicted_emailfrom'] = df['predicted_emailfrom'].apply(lambda x: '' if len(x)!=5 else x)
df['predicted_emailto'] = df['predicted_emailto'].apply(lambda x: '' if len(x)!=5 else x)
df.head(2)

#### Validation of Origin Port Prediction

In [ ]:
# Origin Port UNCODE validation

df.loc[(df['predicted_fromtype'] == 'port'),['predicted_fromtype','predicted_origin',
         'predicted_origin_uncode','predicted_origin_match_score','predicted_origin_port',
                                             'predicted_emailfrom', 'expected_emailfrom']].sort_values(by = 'predicted_origin_match_score').head()

In [ ]:
x = (df[(df['predicted_fromtype'] == 'port') & (df['predicted_emailfrom']==df['expected_emailfrom'])].shape[0]/df[(df['predicted_fromtype'] == 'port')].shape[0])*100
print('Expected and Predicted Origin UNCODES match {}% of the times when the Origin Type is Port'.format(x))

In [ ]:
# Ports where Expected vs Predicted UNCODEs on the Origin side don't match

df.loc[(df['predicted_fromtype'] == 'port') & (df['predicted_emailfrom'] != df['expected_emailfrom']),['emailid','email_processed','predicted_fromtype','predicted_origin',
         'predicted_origin_uncode','predicted_origin_match_score','predicted_origin_port',
                                             'predicted_emailfrom', 'expected_emailfrom']]


#### Valiadation of Destination Port Prediction

In [ ]:
# Ports where Expected vs Predicted UNCODEs on the Destination side don't match

x = (df[(df['predicted_totype'] == 'port')  & (df['predicted_emailto']==df['expected_emailto'])].shape[0]/df[(df['predicted_totype'] == 'port')].shape[0])*100
print('Expected and Predicted Destination UNCODES match {}% of the times when the Destination Type is Port'.format(x))


In [ ]:

df.loc[(df['predicted_totype'] == 'port') & (df['predicted_emailto'] != df['expected_emailto']),['email_processed','predicted_totype','predicted_destination',
         'predicted_destination_uncode','predicted_destination_match_score','predicted_destination_port',
                                             'predicted_emailto', 'expected_emailto']]


**new york, usa	- these kind of case have been dealt below**

In [ ]:
# Analyze the Origin Ports match score

print('% of Origin Ports that have been mapped to a UNCODE: ',(df[(df['predicted_fromtype'] =='port') & (df['predicted_emailfrom'] != '') ].shape[0]/df[(df['predicted_fromtype'] =='port')].shape[0])*100)
# Analyze the Origin Ports match score

print('% of Destination Ports that have been mapped a UNCODE: ',(df[(df['predicted_totype'] =='port') & (df['predicted_emailto'] != '')].shape[0]/df[(df['predicted_totype'] =='port')].shape[0])*100)

#### Exception Handling Technique for Predicted Port Names when one City exists in Different Countries

In [ ]:
df.loc[(df['predicted_fromtype']=='port') & (df['predicted_origin'].str.contains('canada')) &  (df['predicted_origin'].str.contains('vancouver')) ,['predicted_fromtype','predicted_origin','predicted_emailfrom','expected_emailfrom']]

In [ ]:
df[(df['predicted_fromtype'] == 'port') & (df['predicted_emailfrom']==df['expected_emailfrom'])].head(2)

### Exception Handling for Cities that exist in different Countries

In [ ]:
# Fix Canada 

df['predicted_emailfrom'] = np.where((df['predicted_fromtype']=='port') & 
                                    (df['predicted_origin_port'].isin(['USVAN'])), 'CAVAN', df['predicted_emailfrom'])

df['predicted_emailto'] = np.where((df['predicted_totype']=='port')&
                                 (df['predicted_destination_port'].isin(['USVAN'])), 'CAVAN', df['predicted_emailto'])

# Fix New York

df['predicted_emailfrom'] = np.where((df['predicted_fromtype']=='port') & 
                                    (df['predicted_origin_port'].isin(['GBNWY'])), 'USNYC', df['predicted_emailfrom'])

df['predicted_emailto'] = np.where((df['predicted_totype']=='port')&
                                 (df['predicted_destination_port'].isin(['GBNWY'])), 'USNYC', df['predicted_emailto'])

# Fix London

df['predicted_emailfrom'] = np.where((df['predicted_fromtype']=='port') & 
                                    (df['predicted_origin_port'].isin(['ZAELS', 'GBLGP', 'GBLHR','USLDN'])), 'GBLON', df['predicted_emailfrom'])

df['predicted_emailto'] = np.where((df['predicted_totype']=='port')&
                                 (df['predicted_destination_port'].isin(['ZAELS', 'GBLGP', 'GBLHR','USLDN'])), 'GBLON', df['predicted_emailto'])

# Fix Cartagena

df['predicted_emailfrom'] = np.where((df['predicted_fromtype']=='port') & 
                                    (df['predicted_origin_port'].isin(['CLCGN', 'ESCAR'])), 'COCTG', df['predicted_emailfrom'])

df['predicted_emailto'] = np.where((df['predicted_totype']=='port')&
                                 (df['predicted_destination_port'].isin(['CLCGN', 'ESCAR'])), 'COCTG', df['predicted_emailto'])


In [ ]:
# Why is the Origin Prediction Null? - Looks like Prediction Origin is incorrect 

df.loc[(df['predicted_fromtype'] =='port') & (df['predicted_emailfrom'] == ''),['predicted_fromtype','predicted_origin',
         'predicted_origin_uncode','predicted_origin_match_score','predicted_origin_port',
                                             'predicted_emailfrom', 'expected_emailfrom'] ]

In [ ]:
# Why is the Destination Prediction Null ? Probably using only CFS Dest is a better approach

df.loc[(df['predicted_totype'] =='port') & (df['predicted_emailto'] == ''),['predicted_totype','predicted_destination',
         'predicted_destination_uncode','predicted_destination_match_score','predicted_destination_port',
                                             'predicted_emailto', 'expected_emailto']]

In [ ]:
x = (df[(df['predicted_fromtype'] == 'port') & (df['predicted_emailfrom']==df['expected_emailfrom'])].shape[0]/df[(df['predicted_fromtype'] == 'port')].shape[0])*100
print('Expected and Predicted Origin UNCODES match {}% of the times when the Origin Type is Port'.format(x))

In [ ]:
x = (df[(df['predicted_totype'] == 'port')  & (df['predicted_emailto']==df['expected_emailto'])].shape[0]/df[(df['predicted_totype'] == 'port')].shape[0])*100
print('Expected and Predicted Destination UNCODES match {}% of the times when the Destination Type is Port'.format(x))

#### Analysis of Mismatched Predicted vs Expected Ports

In [ ]:
df.loc[(df['predicted_fromtype'] == 'port') & (df['predicted_emailfrom']!=df['expected_emailfrom']),['predicted_fromtype','predicted_origin',
         'predicted_origin_uncode','predicted_origin_match_score','predicted_origin_port',
                                             'predicted_emailfrom', 'expected_emailfrom']]


In [ ]:
df.loc[(df['predicted_totype'] == 'port') & (df['predicted_emailto']!=df['expected_emailto']),['email_processed','predicted_totype','predicted_destination',
         'predicted_destination_uncode','predicted_destination_match_score','predicted_destination_port',
                                             'predicted_emailto', 'expected_emailto']]

In [ ]:
df.rename(columns = {'predicted_emailto':'predicted_port_destination',
                    'predicted_emailfrom':'predicted_port_origin'}, inplace = True) 

<h1><center> 2. Door Booking Cleaning :Extract City, Zip and Country ( Origin + Destination ) </h1></center>

### Method 2. : Use ChatGPT

In [ ]:
def door_address(location):
    prompt = f"""
    Extract city, zipcode and country from the given location.
    
    City: [city]
    Zipcode:[zipcode]
    Country:[country]
        
    The response should include only 3 lines with 3 fields mentioned above in the same order. Return fields with "None" if there is insufficient information.
    
    Review: ```{location}```
    """
    
    response = get_completion(prompt)
    return response

In [ ]:
df['predicted_dest_zip_match'] = np.where(df['predicted_destination'].str.contains('\d') == True, 'Zip Found','Not Found')
df['predicted_origin_zip_match'] = np.where(df['predicted_origin'].str.contains('\d') == True, 'Zip Found','Not Found')

In [ ]:
origin_door_columns = ['predicted_origin_city_zipcode_country']
destination_door_columns = ['predicted_destination_city_zipcode_country']
df[origin_door_columns] = ['']
df[destination_door_columns] = ['']

for i, value in enumerate(df['email_processed']):
    fromtype = df.loc[i, 'predicted_fromtype']
    totype = df.loc[i, 'predicted_totype']
    predicted_origin = df.loc[i, 'predicted_origin']
    predicted_destination = df.loc[i, 'predicted_destination']
    predicted_origin_zip_match = df.loc[i, 'predicted_origin_zip_match']
    predicted_dest_zip_match = df.loc[i, 'predicted_dest_zip_match']
    
    if (('door' in fromtype.lower()) | ('Zip Found' in predicted_origin_zip_match))  :
        result_origin = door_address(predicted_origin)
        df.loc[i, origin_door_columns] = result_origin
        
    if (('door' in totype.lower()) | ('Zip Found' in predicted_dest_zip_match)) :
        result_destination = door_address(predicted_destination)
        df.loc[i, destination_door_columns] = result_destination

#### Parse, Clean and Analyse the Predicted City, Zip and Country ( Origin )

In [ ]:
df['predicted_origin_city'] = df['predicted_origin_city_zipcode_country'].str.split('\n').str[0].str.replace('City: ','').replace('',np.nan).str.strip().fillna('None')
df['predicted_origin_zipcode'] = df['predicted_origin_city_zipcode_country'].str.split('\n').str[1].str.replace('Zipcode: ','').replace('',np.nan).str.strip().fillna('None')
df['predicted_origin_country'] = df['predicted_origin_city_zipcode_country'].str.split('\n').str[2].str.replace('Country: ','').replace('',np.nan).str.strip().fillna('None')

In [ ]:
df['predicted_fromtype'].unique()

In [ ]:
#### Check for null values

print(df.loc[(df['predicted_fromtype'] == 'door') & ((df['predicted_origin_city'] == 'None') | 
   (df['predicted_origin_zipcode'] == 'None') | (df['predicted_origin_country'] == 'None')) , 
  ['predicted_origin_city_zipcode_country', 'predicted_origin_city', 
    'predicted_origin_zipcode', 'predicted_origin_country','expected_emailfrom']].shape[0])

In [ ]:
df.loc[(df['predicted_fromtype'] == 'door') & ((df['predicted_origin_city'] == 'None') | 
   (df['predicted_origin_zipcode'] == 'None') | (df['predicted_origin_country'] == 'None')), 
  ['email_processed','predicted_origin', 'predicted_origin_city', 
    'predicted_origin_zipcode', 'predicted_origin_country','expected_emailfrom']]

#### Parse, Clean and Analyse the Predicted City, Zip and Country ( Door )

In [ ]:
df['predicted_destination_city'] = df['predicted_destination_city_zipcode_country'].str.split('\n').str[0].str.replace('City: ','').replace('',np.nan).str.strip().fillna('None')
df['predicted_destination_zipcode'] = df['predicted_destination_city_zipcode_country'].str.split('\n').str[1].str.replace('Zipcode: ','').replace('',np.nan).str.strip().fillna('None')
df['predicted_destination_country'] = df['predicted_destination_city_zipcode_country'].str.split('\n').str[2].str.replace('Country: ','').replace('',np.nan).str.strip().fillna('None')

In [ ]:
#### Check for null values

print(   df.loc[ (df['predicted_totype'] == 'door') & ((df['predicted_destination_city'] == 'None') | 
          (df['predicted_destination_zipcode'] == 'None') | 
          (df['predicted_destination_country'] == 'None')) , 
          ['email_processed','predicted_destination', 'predicted_destination_city', 
          'predicted_destination_zipcode', 'predicted_destination_country','expected_emailto']].shape[0])

In [ ]:
df.loc[(df['predicted_totype'] == 'door') & ((df['predicted_destination_city'] == 'None') | 
   (df['predicted_destination_zipcode'] == 'None') | (df['predicted_destination_country'] == 'None')), 
  ['email_processed','predicted_destination', 'predicted_destination_city', 
    'predicted_destination_zipcode', 'predicted_destination_country','expected_emailto']]

In [ ]:
df['predicted_destination_country'] = df['predicted_destination_country'].str.strip().str.lower()
df['predicted_origin_country'] = df['predicted_origin_country'].str.strip().str.lower()

df['predicted_destination_zipcode'] = df['predicted_destination_zipcode'].str.strip().str.lower()
df['predicted_origin_zipcode'] = df['predicted_origin_zipcode'].str.strip().str.lower()

df['predicted_destination_city'] = df['predicted_destination_city'].str.strip().str.lower()
df['predicted_origin_city'] = df['predicted_origin_city'].str.strip().str.lower()

#### Exception Handling

#### Scenario 1 :  When a territory has been predicted instead of the Country

In [ ]:
df['predicted_origin_country'].unique()

In [ ]:
df['predicted_destination_country'].unique()

In [ ]:
# Change to the required Country Format

df['predicted_destination_country'] = df['predicted_destination_country'].str.lower().str.replace('united states', 'u.s.a.')
df['predicted_origin_country'] = df['predicted_origin_country'].str.lower().str.replace('united states', 'u.s.a.')

df['predicted_destination_country'] = df['predicted_destination_country'].str.lower().str.replace('taiwan', 'taiwan, china')
df['predicted_origin_country'] = df['predicted_origin_country'].str.lower().str.replace('taiwan', 'taiwan, china')

df['predicted_destination_country'] = df['predicted_destination_country'].str.lower().str.replace('uk', 'united kingdom')
df['predicted_origin_country'] = df['predicted_origin_country'].str.lower().str.replace('uk', 'united kingdom')


In [ ]:
df['predicted_origin_country'] = np.where(df['predicted_origin_country'].str.lower().isin(country_list), df['predicted_origin_country'], 'none')
df['predicted_destination_country'] = np.where(df['predicted_destination_country'].str.lower().isin(country_list), df['predicted_destination_country'], 'none')


In [ ]:
df['predicted_origin_country'].unique()

In [ ]:
df['predicted_destination_country'].unique()

#### Scenario 2 : When a territory has been predicted instead of the Zipcode

In [ ]:
df['predicted_destination_zipcode'] = np.where(df['predicted_destination_zipcode'].str.contains('\d') == True, df['predicted_destination_zipcode'], 'none')
df['predicted_origin_zipcode'] = np.where(df['predicted_origin_zipcode'].str.contains('\d') == True, df['predicted_origin_zipcode'], 'none')

In [ ]:
df['predicted_origin_zipcode'].unique()

In [ ]:
df['predicted_destination_zipcode'].unique()


## Method 2 : Use Inbuilt Geopy Library to determine the null values

In [ ]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="https")

In [ ]:
df['rev_predicted_destination'] = df['predicted_destination_city'] + ' ' + df['predicted_destination_zipcode'] + ' ' + df['predicted_destination_country']
df['rev_predicted_origin'] = df['predicted_origin_city'] + ' ' + df['predicted_origin_zipcode'] + ' ' + df['predicted_origin_country']

In [ ]:
df['rev_predicted_destination'] = df['rev_predicted_destination'].str.replace('none','').str.strip()
df['rev_predicted_origin'] = df['rev_predicted_origin'].str.replace('none','').str.strip()

In [ ]:
# df[['rev_predicted_destination','predicted_destination','predicted_destination_city','predicted_destination_zipcode','predicted_destination_country']]

#### Origin Side 

In [ ]:
%%time
df['predicted_orig_geoaddress'] = df['rev_predicted_origin'].str.lower().apply(lambda x: geolocator.geocode(x,  language='en')).astype(str)
df['predicted_orig_geoaddress'] = df['predicted_orig_geoaddress'].str.lower().str.strip()
df['predicted_origin_zipcode1'] = df['predicted_orig_geoaddress'].str.split(',').str[-2].str.strip()
df['predicted_origin_country1'] = df['predicted_orig_geoaddress'].str.split(',').str[-1].str.strip()


#### Destination Side

In [ ]:
%%time
df['predicted_dest_geoaddress'] = df['rev_predicted_destination'].str.lower().apply(lambda x: geolocator.geocode(x,  language='en')).astype(str)
df['predicted_dest_geoaddress'] = df['predicted_dest_geoaddress'].str.lower().str.strip()
df['predicted_destination_zipcode1'] = df['predicted_dest_geoaddress'].str.split(',').str[-2].str.strip()
df['predicted_destination_country1'] = df['predicted_dest_geoaddress'].str.split(',').str[-1].str.strip()

#### Cleaning Country and Zip on both sides as done above

- Country

In [ ]:
df['predicted_origin_country1'].unique()

In [ ]:
df['predicted_destination_country1'].unique()

In [ ]:
df['predicted_origin_country1'] = df['predicted_origin_country1'].str.replace('united states', 'u.s.a.')
df['predicted_origin_country1'] = np.where(df['predicted_origin_country1'].isin(country_list), df['predicted_origin_country1'], 'none')

df['predicted_origin_country1'] = df['predicted_origin_country1'].str.replace('taiwan', 'taiwan, china')
df['predicted_origin_country1'] = np.where(df['predicted_origin_country1'].isin(country_list), df['predicted_origin_country1'], 'none')

df['predicted_origin_country1'] = df['predicted_origin_country1'].str.replace('uk', 'united kingdom')
df['predicted_origin_country1'] = np.where(df['predicted_origin_country1'].isin(country_list), df['predicted_origin_country1'], 'none')

In [ ]:
df['predicted_destination_country1'] = df['predicted_destination_country1'].str.replace('united states', 'u.s.a.')
df['predicted_destination_country1'] = np.where(df['predicted_destination_country1'].isin(country_list), df['predicted_destination_country1'], 'none')

df['predicted_destination_country1'] = df['predicted_destination_country1'].str.replace('taiwan', 'taiwan, china')
df['predicted_destination_country1'] = np.where(df['predicted_destination_country1'].isin(country_list), df['predicted_destination_country1'], 'none')

df['predicted_destination_country1'] = df['predicted_destination_country1'].str.replace('uk', 'united kingdom')
df['predicted_destination_country1'] = np.where(df['predicted_destination_country1'].isin(country_list), df['predicted_destination_country1'], 'none')

In [ ]:
df['predicted_destination_country1'].unique()

- Zipcode

In [ ]:
df['predicted_destination_zipcode1'] = np.where(df['predicted_destination_zipcode1'].str.contains('\d') == True, df['predicted_destination_zipcode1'], 'none')
df['predicted_origin_zipcode1'] = np.where(df['predicted_origin_zipcode1'].str.contains('\d') == True, df['predicted_origin_zipcode1'], 'none')

In [ ]:
df['predicted_destination_zipcode1'].unique()

In [ ]:
df['predicted_origin_zipcode1'].unique()

In [ ]:
# Fill Zip on the Origin and Destination sides

df['predicted_origin_zipcode'] = np.where( ((df['predicted_fromtype'] == 'door') | (df['predicted_origin_zip_match'] == 'Zip Found')) & (df['predicted_origin_country'] !='none') & (df['predicted_origin_city']!='none')
         & (df['predicted_origin_zipcode']=='none'), df['predicted_origin_zipcode1'], df['predicted_origin_zipcode'])

# Fill Zip on the Destination side 

df['predicted_destination_zipcode'] = np.where( ((df['predicted_totype'] == 'door') | (df['predicted_dest_zip_match'] == 'Zip Found')) & (df['predicted_destination_country'] !='none') & (df['predicted_destination_city']!='none')
          & (df['predicted_destination_zipcode']=='none'), df['predicted_destination_zipcode1'], df['predicted_destination_zipcode'])

In [ ]:
# Fill Country on the Origin side

df['predicted_origin_country'] = np.where( ((df['predicted_fromtype'] == 'door') | (df['predicted_origin_zip_match'] == 'Zip Found')) & (df['predicted_origin_city'] !='none') & (df['predicted_origin_zipcode']!='none')
        & (df['predicted_origin_country']=='none'), df['predicted_origin_country1'], df['predicted_origin_country'])

# Fill Country on the Destination side

df['predicted_destination_country'] = np.where( ((df['predicted_totype'] == 'door') | (df['predicted_dest_zip_match'] == 'Zip Found')) & (df['predicted_destination_city'] !='none') & (df['predicted_destination_zipcode']!='none')
        & (df['predicted_destination_country']=='none'), df['predicted_destination_country1'], df['predicted_destination_country'])

In [ ]:
df[df['predicted_destination'].str.contains('tainan')][['predicted_totype','predicted_destination','predicted_dest_zip_match','predicted_destination_city','predicted_destination_zipcode','predicted_destination_country']]

In [ ]:
df['predicted_destination_city'].unique()

### Finally : Check the remaining Null cases and use ChatGPT to solve for Country

In [ ]:
df.loc[(df['predicted_fromtype'] == 'door') & ((df['predicted_origin_city'] == 'none') | 
   (df['predicted_origin_zipcode'] == 'None') | (df['predicted_origin_country'] == 'none')), 
  ['email_processed','predicted_origin', 'predicted_origin_city', 
    'predicted_origin_zipcode', 'predicted_origin_country','expected_emailfrom']]

In [ ]:
df.loc[(df['predicted_totype'] == 'door') & ((df['predicted_destination_city'] == 'none') | 
   (df['predicted_destination_zipcode'] == 'none') | (df['predicted_destination_country'] == 'none')), 
  ['email_processed', 'rev_predicted_destination', 'predicted_destination', 'predicted_destination_city', 
    'predicted_destination_zipcode', 'predicted_destination_country1','expected_emailto']]

In [ ]:
# df['rev_predicted_destination'] = df['predicted_destination_city'] + ' ' + df['predicted_destination_zipcode']
# df['rev_predicted_origin'] = df['predicted_origin_city'] + ' ' + df['predicted_origin_zipcode']

In [ ]:
def country_address(location):
    prompt = f"""
    Return the Country
    
    Country:[country]
        
    Return fields with "None" if there is insufficient information.
    
    Review: ```{location}```
    """
    
    response = get_completion(prompt)
    return response

In [ ]:
df['predicted_dest_country_chatgpt'] = df['predicted_destination'].apply(country_address)
df['predicted_dest_country_chatgpt'] = df['predicted_dest_country_chatgpt'].str.replace('Country: ', '').str.strip().str.lower()
df['predicted_dest_country_chatgpt'].head()

In [ ]:
df['predicted_orig_country_chatgpt'] = df['predicted_origin'].apply(country_address)
df['predicted_orig_country_chatgpt'] = df['predicted_orig_country_chatgpt'].str.replace('Country: ', '').str.strip().str.lower()
# add a line if not in country list return none
# maybe split country , city , zip question within the door address function itself
df['predicted_orig_country_chatgpt'].head()

In [ ]:
# Fill Country on the Origin side

df['predicted_origin_country'] = np.where( ((df['predicted_fromtype'] == 'door') | (df['predicted_origin_zip_match'] == 'Zip Found')) & (df['predicted_origin_city'] !='none') & (df['predicted_origin_zipcode']!='none')
        & (df['predicted_origin_country']=='none'),df['predicted_orig_country_chatgpt'], df['predicted_origin_country'])

# Fill Country on the Destination side

df['predicted_destination_country'] = np.where( ((df['predicted_totype'] == 'door') | (df['predicted_dest_zip_match'] == 'Zip Found')) & (df['predicted_destination_city'] !='none') & (df['predicted_destination_zipcode']!='none')
        & (df['predicted_destination_country']=='none'), df['predicted_dest_country_chatgpt'], df['predicted_destination_country'])

In [ ]:
df.loc[(df['predicted_fromtype'] == 'door') & ((df['predicted_origin_city'] == 'none') | 
   (df['predicted_origin_zipcode'] == 'None') | (df['predicted_origin_country'] == 'none')), 
  ['email_processed','predicted_origin', 'predicted_origin_city', 
    'predicted_origin_zipcode', 'predicted_origin_country','expected_emailfrom']]

In [ ]:
df.loc[(df['predicted_totype'] == 'door') & ((df['predicted_destination_city'] == 'none') | 
   (df['predicted_destination_zipcode'] == 'none') | (df['predicted_destination_country'] == 'none')), 
  ['email_processed', 'rev_predicted_destination', 'predicted_destination', 'predicted_destination_city', 
    'predicted_destination_zipcode', 'predicted_destination_country1','expected_emailto']]

#### Validation 

In [ ]:
df.loc[df['predicted_destination'].str.contains('anchorage, ak 99516'),['predicted_totype','predicted_destination_city','predicted_destination_zipcode','predicted_destination_country','predicted_destination_country1']]

In [ ]:
df.loc[df['predicted_destination'].str.contains('tainan'),['predicted_totype','predicted_destination_city','predicted_destination_zipcode','predicted_destination_country','predicted_destination_country1']]

* We were able to extract these cases earlier - WEST VANCOUVER - V7W - CANADA - we have to include keyword dap in prompt.

**Post Processing for zip code**

In [ ]:
df['predicted_destination_country'] = df['predicted_destination_country'].str.strip()
df['predicted_destination_city'] = df['predicted_destination_city'].str.strip()
df['predicted_destination_zipcode'] = df['predicted_destination_zipcode'].str.strip()

df['predicted_origin_zipcode'] = df['predicted_origin_zipcode'].str.strip()
df['predicted_origin_city'] = df['predicted_origin_city'].str.strip()
df['predicted_origin_country'] = df['predicted_origin_country'].str.strip()

df['predicted_destination_country'] = df['predicted_destination_country'].str.lower()
df['predicted_origin_country'] = df['predicted_origin_country'].str.lower()

In [ ]:
# # access only first part of the zipcode

first_part = ['canada','united kingdom','uk', 'bc', 'on', 'qc', 'on', 'bc', 'ab', 'mb']
canada = ['canada', 'bc', 'on', 'qc', 'mb', 'ab']
no_space = ['netherlands','czech republic']
no_zipcode = ['bahrain','chile','el salvador','ghana','hong kong','jordan','malaysia','morocco','pakistan','panama','qatar','saudi arabia','sri lanka','u.a.e.','ukraine','vietnam', ' u.a.e.']

In [ ]:
df.loc[df['predicted_origin_country'].str.lower().isin(first_part),'predicted_origin_zipcode']=df['predicted_origin_zipcode'].str.split(' ').str[0]
df.loc[df['predicted_origin_country'].str.lower().isin(canada),'predicted_origin_zipcode']=df['predicted_origin_zipcode'].str.split('-').str[0]
df.loc[df['predicted_origin_country'].str.lower().isin(canada),'predicted_origin_zipcode']=df['predicted_origin_zipcode'].str[:3]
df.loc[df['predicted_origin_country'].str.lower().isin(no_space),'predicted_origin_zipcode']=df['predicted_origin_zipcode'].str.replace(' ','')
df.loc[df['predicted_origin_country'].str.lower().isin(no_zipcode),'predicted_origin_door_location']=  df['predicted_origin_city'] + ' - ' + df['predicted_origin_country']

df['predicted_destination_country'] = df['predicted_destination_country'].str.lower()
df.loc[df['predicted_destination_country'].str.lower().isin(first_part),'predicted_destination_zipcode']=df['predicted_destination_zipcode'].str.split(' ').str[0]
df.loc[df['predicted_destination_country'].str.lower().isin(canada),'predicted_destination_zipcode']=df['predicted_destination_zipcode'].str.split('-').str[0]
df.loc[df['predicted_destination_country'].str.lower().isin(canada),'predicted_destination_zipcode']=df['predicted_destination_zipcode'].str[:3]
df.loc[df['predicted_destination_country'].str.lower().isin(no_space),'predicted_destination_zipcode']=df['predicted_destination_zipcode'].str.replace(' ','')
df.loc[df['predicted_destination_country'].str.lower().isin(no_zipcode),'predicted_destination_door_location']=  df['predicted_destination_city'] + ' - ' + df['predicted_destination_country']

In [ ]:
df['predicted_origin_country'] = df['predicted_origin_country'].astype(str)

In [ ]:
df['predicted_origin_zipcode'].unique()

In [ ]:
df['predicted_origin_country'].unique()

In [ ]:
df['predicted_origin_zipcode'] = df['predicted_origin_zipcode'].str.replace('non', 'None')
df['predicted_origin_zipcode'] = df['predicted_origin_zipcode'].str.replace('nonee', 'None')
df['predicted_origin_zipcode'] = df['predicted_origin_zipcode'].fillna('None')
df['predicted_origin_city'] = df['predicted_origin_city'].fillna('None')
df['predicted_origin_country'] = df['predicted_origin_country'].fillna('none')

In [ ]:
df['predicted_destination_zipcode'] = df['predicted_destination_zipcode'].str.replace('non', 'None')
df['predicted_destination_zipcode'] = df['predicted_destination_zipcode'].str.replace('nonee', 'None')
df['predicted_destination_zipcode'] = df['predicted_destination_zipcode'].fillna('None')
df['predicted_destination_city'] = df['predicted_destination_city'].fillna('None')
df['predicted_destination_country'] = df['predicted_destination_country'].fillna('none')

In [ ]:
# df = temp.copy()

In [ ]:
df.loc[df['predicted_destination'].str.contains('anchorage, ak 99516'),['predicted_destination_country']]

### Create a Single column for Predicted Origin and Predicted Destination respectively

In [ ]:
df['predicted_door_destination'] = df['predicted_destination_city'].str.strip().str.upper() + ' - ' + df['predicted_destination_zipcode'].str.strip().str.upper() + ' - '+ df['predicted_destination_country'].str.strip().str.upper()
df['predicted_door_origin'] = df['predicted_origin_city'].str.strip().str.upper() + ' - ' + df['predicted_origin_zipcode'].str.strip().str.upper() + ' - '+ df['predicted_origin_country'].str.strip().str.upper()

In [ ]:
df.loc[df['predicted_destination'].str.contains('anchorage, ak 99516'),['predicted_door_destination','predicted_destination_country']]

In [ ]:
#### When Origin is a Door

df.loc[:,['predicted_totype','predicted_origin','predicted_destination','predicted_port_destination','predicted_door_destination','expected_emailto']]#.head()


In [ ]:
#### When Origin is a Port 

df.loc[:,['predicted_fromtype','predicted_origin','predicted_destination','predicted_port_origin','predicted_door_origin','expected_emailfrom']]#.head()


In [ ]:
 # Set Final Predicted Destination column 

df['predicted_Destination_Final'] = np.where( ((df['predicted_totype'] == 'door') ) , df['predicted_door_destination'], df['predicted_port_destination'])

# Set Final Predicted Origin column # 

df['predicted_Origin_Final'] = np.where( ((df['predicted_fromtype'] == 'door') ) , df['predicted_door_origin'], df['predicted_port_origin'])

In [ ]:
df.rename(columns = {'expected_emailto':'expected_Destination_Final',
                     'expected_emailfrom':'expected_Origin_Final'}, inplace = True) 

In [ ]:
# Validating Final Destination

df.loc[:,['predicted_totype','predicted_destination', 'predicted_destination_port','predicted_door_destination','predicted_Destination_Final','expected_Destination_Final']].head()


In [ ]:
# # Validating Final Origin

df.loc[:,['predicted_fromtype','predicted_origin','predicted_origin_port','predicted_door_origin','predicted_Origin_Final','expected_Origin_Final']].head()


In [ ]:
df[[ 'predicted_Origin_Final',
   'expected_Origin_Final', 'predicted_Destination_Final','expected_Destination_Final' ]] = df[[ 'predicted_Origin_Final',
                                                                 'expected_Origin_Final','predicted_Destination_Final','expected_Destination_Final']].apply(lambda x: x.str.strip().str.upper())

In [ ]:
df.loc[df['predicted_Origin_Final'].str.contains('- NONE -') 
       | df['predicted_Origin_Final'].str.contains('NONE -')
       | df['predicted_Origin_Final'].str.contains('- NONE')
       | df['predicted_Origin_Final'].str.contains('- NON -') 
       | df['predicted_Origin_Final'].str.contains('NON -')
       | df['predicted_Origin_Final'].str.contains('- NON'),['predicted_Origin_Final','predicted_destination_uncode']]

<h1><center> Compute the Accuracy of the Origin + Destination UNCODEs/Addresses </h1></center>

In [ ]:
df_output = df.copy()

In [ ]:
df_output[[ 'predicted_Origin_Final',
   'expected_Origin_Final', 'predicted_Destination_Final','expected_Destination_Final' ]] =df_output[[ 'predicted_Origin_Final',
   'expected_Origin_Final', 'predicted_Destination_Final','expected_Destination_Final' ]].apply(lambda x : x.str.strip())

In [ ]:
# #### Harcode Changes to expected temporarily

df_output['expected_Origin_Final'] = df_output['expected_Origin_Final'].str.replace('CALGARY - T2G3C1 - AB','CALGARY - T2G - CANADA' )
df_output['expected_Origin_Final'] = df_output['expected_Origin_Final'].str.replace('DARLINGTON - DL1 - UK','DARLINGTON - DL1 - UNITED KINGDOM')
df_output['expected_Origin_Final'] = df_output['expected_Origin_Final'].str.replace('MONTOIR DE BRETAGNE - 44550 - FRANCE','MONTOIR-DE-BRETAGNE - 44550 - FRANCE' )
df_output['expected_Origin_Final'] = df_output['expected_Origin_Final'].str.replace('MIAMI - 33147 - FLORIDA','MIAMI - 33147 - U.S.A.' )
# df_output['expected_Origin_Final'] = df_output['expected_Origin_Final'].str.replace('SAINT HUBERT - J3Y - QC','SAINT-HUBERT - J3Y - CANADA' )
# df_output['expected_Origin_Final'] = df_output['expected_Origin_Final'].str.replace('SAINTE CATHERINE - J5C - CANADA','SAINTE-CATHERINE - J5C - CANADA' )
# df_output['expected_Origin_Final'] = df_output['expected_Origin_Final'].str.replace('LEDUC (NISKU) - T9E - CANADA','LEDUC - T9E - CANADA' )
# df_output['expected_Origin_Final'] = df_output['expected_Origin_Final'].str.replace('ANCHORAGE - 99516 - U.S.A','ANCHORAGE - 99516 - U.S.A.')
df_output['expected_Origin_Final'] = df_output['expected_Origin_Final'].str.replace('BRADFORD ON - L3Z - CANADA','BRADFORD - L3Z - CANADA')
df_output['expected_Origin_Final'] = df_output['expected_Origin_Final'].str.replace('SUNDERN - 59846 - GERMANY','SUNDERN-AMECKE - 59846 - GERMANY')
df_output['expected_Origin_Final'] = df_output['expected_Origin_Final'].str.replace('SAINTE CLAIRE - G0R - CANADA','SAINTE-CLAIRE - G0R - CANADA')
df_output['expected_Origin_Final'] = df_output['expected_Origin_Final'].str.replace('TAINAN CITY - 70841 - TAIWAN','TAINAN - 70841 - TAIWAN')

In [ ]:
# #### Harcode Changes to expected temporarily

df_output['expected_Destination_Final'] = df_output['expected_Destination_Final'].str.replace('MONTOIR-DE-BRETAGNE - 44550 - FRANCE','MONTOIR DE BRETAGNE - 44550 - FRANCE' )
df_output['expected_Destination_Final'] = df_output['expected_Destination_Final'].str.replace('DARLINGTON - DL1 - UK','DARLINGTON - DL1 - UNITED KINGDOM')
df_output['expected_Destination_Final'] = df_output['expected_Destination_Final'].str.replace('TAINAN - 70841 - TAIWAN','TAINAN CITY - 70841 - TAIWAN' )
df_output['expected_Destination_Final'] = df_output['expected_Destination_Final'].str.replace('MONTOIR DE BRETAGNE - 44550 - FRANCE','MONTOIR-DE-BRETAGNE - 44550 - FRANCE' )
# df_output['expected_Destination_Final'] = df_output['expected_Destination_Final'].str.replace( 'SAINT HUBERT - J3Y - QC','SAINT-HUBERT - J3Y - CANADA')
# df_output['expected_Destination_Final'] = df_output['expected_Destination_Final'].str.replace('SAINTE-CATHERINE - J5C - CANADA','SAINTE CATHERINE - J5C - CANADA')
# df_output['expected_Destination_Final'] = df_output['expected_Destination_Final'].str.replace('LEDUC (NISKU) - T9E - CANADA','LEDUC - T9E - CANADA')
# df_output['expected_Destination_Final'] = df_output['expected_Destination_Final'].str.replace('MUNICH - 81669 - GERMANY','MÜNCHEN - 81669 - GERMANY')
df_output['expected_Destination_Final'] = df_output['expected_Destination_Final'].str.replace('MIAMI - 33147 - FLORIDA','MIAMI - 33147 - U.S.A.' )
df_output['expected_Destination_Final'] = df_output['expected_Destination_Final'].str.replace('BRADFORD ON - L3Z - CANADA','BRADFORD - L3Z - CANADA')
df_output['expected_Destination_Final'] = df_output['expected_Destination_Final'].str.replace('SAINTE CLAIRE - G0R - CANADA','SAINTE-CLAIRE - G0R - CANADA')
df_output['expected_Destination_Final'] = df_output['expected_Destination_Final'].str.replace('TAINAN CITY - 70841 - TAIWAN','TAINAN - 70841 - TAIWAN')

In [ ]:
#df_output['expected_Destination_Final'] = df_output['expected_Destination_Final'].str.replace( 'SAINT HUBERT - J3Y - QC','SAINT-HUBERT - J3Y - CANADA')

In [ ]:
df_output['predicted_Destination_Final'] = df_output['predicted_Destination_Final'].str.replace('UNITED STATES','U.S.A.')
df_output['predicted_Origin_Final'] = df_output['predicted_Origin_Final'].str.replace('UNITED STATES','U.S.A.')

In [ ]:
df_output['predicted_Destination_Final'] = df_output['predicted_Destination_Final'].str.replace('UNITED STATES','U.S.A.')
df_output['predicted_Origin_Final'] = df_output['predicted_Origin_Final'].str.replace('UNITED STATES','U.S.A.')

In [ ]:
df_output[[ 'predicted_Origin_Final',
   'expected_Origin_Final', 'predicted_Destination_Final','expected_Destination_Final' ]].isnull().sum()

#### Create an indicator to determine full or partial lane match

In [ ]:
df_output['Lane_Match_Status'] = np.where((df_output['predicted_Origin_Final'] == df_output['expected_Origin_Final']) & 
                             (df_output['predicted_Destination_Final'] == df_output['expected_Destination_Final']) , 
                             'Full Match', 'Partail Match')

In [ ]:
df_output['Origin_Match_Status'] = np.where(
    (df_output['predicted_Origin_Final'] == df_output['expected_Origin_Final']) , 'Match', 'Not Matched')

In [ ]:
df_output['Destination_Match_Status'] = np.where(
         (df_output['predicted_Destination_Final'] == df_output['expected_Destination_Final']), 'Match', 'Not Matched')

#### 1. Origin Match %

In [ ]:
a  = (df_output[df_output['predicted_Origin_Final'] == df_output['expected_Origin_Final']].shape[0]/df_output.shape[0])*100
print('Origin Matches {}% of times :'.format(a))

In [ ]:
# Analyze the Mismatched entries

df_output.loc[(df_output['Origin_Match_Status'] == 'Not Matched'),['emailid','email_processed','predicted_fromtype','predicted_origin','predicted_origin_port','predicted_door_origin','predicted_Origin_Final','expected_Origin_Final','predicted_origin_uncode']]


#### 2. Destination Match %

In [ ]:
b  = (df_output[df_output['predicted_Destination_Final'] == df_output['expected_Destination_Final']].shape[0]/df_output.shape[0])*100
print('Destination Matches {}% of times :'.format(b))

In [ ]:
df.loc[df['email_processed'].str.contains('elmira'),['predicted_fromtype','predicted_origin','predicted_origin_zipcode', 'predicted_origin_zipcode1']]

In [ ]:
# Analyze the Mismatched entries

df_output.loc[(df_output['Destination_Match_Status'] == 'Not Matched'),['emailid','email_processed','predicted_totype','expected_Destination_type','predicted_destination','predicted_destination_port','predicted_door_destination','predicted_Destination_Final','expected_Destination_Final','predicted_destination_uncode']]


In [ ]:
# geolocator.geocode('CORNWALL K6J 3E6').raw

In [ ]:
# Analyze the Mismatched entries

df_output.loc[df_output['Destination_Match_Status'] == 'Not Matched',['emailid','email_processed','predicted_totype','expected_Destination_type','predicted_destination','predicted_destination_port','predicted_door_destination','predicted_Destination_Final','expected_Destination_Final','predicted_destination_uncode']]['email_processed'].to_excel('test.xlsx')


#### 3. Lane Match % 

In [ ]:
c  = (df_output[(df_output['predicted_Origin_Final'] == df_output['expected_Origin_Final']) & 
         (df_output['predicted_Destination_Final'] == df_output['expected_Destination_Final'])].shape[0]/df_output.shape[0])*100
print('Entire Lane Matches {}% of times :'.format(c))

In [ ]:
df_output[(df_output['predicted_Origin_Final'] == df_output['expected_Origin_Final']) & 
         (df_output['predicted_Destination_Final'] == df_output['expected_Destination_Final'])].shape

* 22 P2P
* 16 involve door locations -> zipcode

In [ ]:
df_output.loc[df['predicted_destination'].str.contains('anchorage, ak 99516'),['predicted_Destination_Final','predicted_door_destination','predicted_destination_country']]

In [ ]:
print('Origin Address/Port Matches {}% of times'.format(round(a,2)))
print('Destination Address/Port Matches {}% of times'.format(round(b,2)))
print('Entire Lane Matches {}% of times'.format(round(c,2)))

<h1><center> Analyze the intermediate output </h1></center>

In [ ]:
df_output['expected_Destination_type'].unique()

In [ ]:
df_output['expected_Origin_type'].unique()

In [ ]:
df_output['expected_Destination_type'] = np.where(df_output['expected_Destination_type']  == '1', 'port', 'door')
df_output['expected_Origin_type'] = np.where(df_output['expected_Origin_type']  == '1', 'port', 'door')

In [ ]:
df_output[['emailid','email_processed','predicted_fromtype', 'expected_Origin_type','predicted_origin','predicted_origin_port','predicted_door_origin','predicted_Origin_Final','expected_Origin_Final','predicted_origin_uncode',
  'predicted_totype','expected_Destination_type','predicted_destination','predicted_destination_port','predicted_door_destination','predicted_Destination_Final','expected_Destination_Final','predicted_destination_uncode',
  'Origin_Match_Status','Destination_Match_Status' ,'Lane_Match_Status']].to_excel('EA_Output_21stAug.xlsx')


<h1><center> Binary Metrics </h1></center>

In [18]:
df = pd.read_pickle('Processed_EmailOutput21stAugust.pkl')
df.head()

,Unnamed: 0,body,emailid,Email,email_processed,expected_Origin,expected_Origin_Final,expected_Origin_type,expected_Destination,expected_Destination_Final,expected_Destination_type,expected_Weight,expected_Dimensions,expected_Volume,expected_Quantity,expected_Isstackable,expected_Ishazardous,expected_Istoploaded,expected_Response Status,Exclusion Indicator,Comments_BG,Comments_BG_Binary,predicted_class,email_processed_revised,predicted_origin_destination,predicted_origin,predicted_destination,predicted_fromtype,predicted_totype,OriginType_Match,DestinationType_Match,predicted_origin_match,predicted_origin_match_port,predicted_origin_match_score,predicted_origin_match_junk,predicted_destination_match,predicted_destination_match_port,predicted_destination_match_score,predicted_destination_match_junk,Port_x,predicted_origin_uncode,Port_y,predicted_destination_uncode,predicted_origin_port,predicted_destination_port,predicted_port_destination,predicted_port_origin,predicted_dest_zip_match,predicted_origin_zip_match,predicted_origin_city_zipcode_country,predicted_destination_city_zipcode_country,predicted_origin_city,predicted_origin_zipcode,predicted_origin_country,predicted_destination_city,predicted_destination_zipcode,predicted_destination_country,rev_predicted_destination,rev_predicted_origin,predicted_orig_geoaddress,predicted_origin_zipcode1,predicted_origin_country1,predicted_dest_geoaddress,predicted_destination_zipcode1,predicted_destination_country1,predicted_dest_country_chatgpt,predicted_orig_country_chatgpt,predicted_origin_door_location,predicted_destination_door_location,predicted_door_destination,predicted_door_origin,predicted_Destination_Final,predicted_Origin_Final,Lane_Match_Status,Origin_Match_Status,Destination_Match_Status
0,0,VS: LCL Quote Request | From location : Britis...,Simon Holt <Simon.holt@nordic-on.com>,VS: LCL Quote Request | From location : Britis...,vs: lcl quote request | from location : britis...,british columbia - penticton - v2a - canada,PENTICTON - V2A - CANADA,door,"deham , hamburg , germany",DEHAM,port,nan,nan,nan,nan,nan,nan,nan,No Offers found,Do not exclude,Do not exclude,Do not exclude,Quotation Request,vs: lcl quote request | from location : briti...,British Columbia - Penticton - V2A - Canada\n\...,british columbia - penticton - v2a - canada,"deham, hamburg, germany",door,port,Matched,Matched,"('PALMERSTON NORTH', 42, 567)",PALMERSTON NORTH,42,567,"(HAMBURG, 100, 1827)",HAMBURG,100,1827,PALMERSTON NORTH,NZPMR,HAMBURG,DEHAM,,DEHAM,DEHAM,,Not Found,Zip Found,City: Penticton\nZipcode: V2A\nCountry: Canada,,penticton,v2a,canada,none,Nonee,none,,penticton v2a canada,"penticton, regional district of okanagan-simil...",none,canada,none,none,none,germany,canada,NaN,NaN,NONE - NONEE - NONE,PENTICTON - V2A - CANADA,DEHAM,PENTICTON - V2A - CANADA,Full Match,Match,Match
1,2,Santos**This mail is from outside our organiza...,Linda-Nancy Colizza <Linda-Nancy.Colizza@hellm...,Santos**This mail is from outside our organiza...,"santos good morning, need rate please, termina...",terminal montreal,CAMTR,port,tml santos,BRSSZ,port,2146 kg,44 x44 x42,162624,2.0,nan,No,nan,No Offers found,Do not exclude,Do not exclude,Do not exclude,Quotation Request,"""Santos good morning, need rate please, term...",Terminal Montreal TML Santos\n\nDestination Lo...,terminal montreal tml santos,unknown,port,port,Matched,Matched,"('SANTOS', 100, 1296)",SANTOS,100,1296,"(QUEENSTOWN, 59, 557)",QUEENSTOWN,59,557,SANTOS,BRSSZ,QUEENSTOWN,NZZQN,BRSSZ,,,BRSSZ,Not Found,Not Found,,,none,Nonee,none,none,Nonee,none,,,none,none,none,none,none,none,"i'm sorry, but i cannot determine the country ...",none,NaN,NaN,NONE - NONEE - NONE,NONE - NONEE - NONE,,BRSSZ,Partail Match,Not Matched,Not Matched
2,3,"RFQ / ITN door Winnipeg, MB to CFS Shanghai,...",Sally Chieng <sallyc@itn-logistics.ca>,"RFQ / ITN door Winnipeg, MB to CFS Shanghai,...","rfq / itn door winnipeg, mb to cfs shanghai, c...",95 alexander st. winnipeg mb,WINNIPEG - R3B - CANA

## Hazardous vs Non Hazardous

In [19]:
df['expected_Ishazardous'] = df['expected_Ishazardous'].str.lower()
df['expected_Ishazardous'].value_counts().index

Index(['nan', 'no'], dtype='object')

In [20]:
df['expected_Ishazardous'].value_counts() #.index

nan    39
no     10
Name: expected_Ishazardous, dtype: int64

In [21]:
df.loc[df['expected_Ishazardous']=='no']['email_processed'].values[9]

'ocean rate request - lcl (door delivery) good afternoon!if you have door service, please quote the below shipment to door. if not, please quote to anchorage port only. can you please provide an lcl rate quotation based on the following specifications (to door consignee as listed): origin: toronto, on destination: anchorage, ak 99516 commodity: plastic recycling containers(non-hazardous) 5 skids at 1,212.0 kg 16.122 cbm 4 x 48x48x84 1 x 52x48x84 please ensure you provide all charges (excluding only customs formalities), as well as transit time, and cut off dates/sailing dates. if you have any questions, please do not hesitate to contact me.'

* For non-hazardous, many a times the expected value is mapped based on intuition like "wild rice is non hazardous", "range rover can't be hazardous material". so for materials like these there were no keywords present in the email.

In [22]:
non_haz = ['non dg','non-dg','non - haz','non-haz','non haz',
           'non-hazardous', 'non hazardous', 'hazardous no']

haz = ['dg','haz', 'hazardous']

# 'hazmats' is to be added, haven't seen any case so haven't added

def check_non_binary_in_row_with_space(strings_list, text_row):
    for string in strings_list:
        if(f" {string} " in text_row or f"{string} " in text_row or f" {string}" in text_row):
            return True
    return False

def check_binary_in_row_with_space(strings_list, text_row):
    for string in strings_list:
        if(f" {string} " in text_row or f"{string} " in text_row or f" {string}" in text_row):
            return True
    return False


df['IsHazardousMaterial'] = np.nan

for i, row in df.iterrows():
    email = row['email_processed']
    if(check_non_binary_in_row_with_space(non_haz, email)):
        df.loc[i, 'IsHazardousMaterial'] = False
    elif((df.loc[i, 'IsHazardousMaterial'] != False)&(check_binary_in_row_with_space(haz, email))):
        df.loc[i, 'IsHazardousMaterial'] = True
    else:
        df.loc[i, 'IsHazardousMaterial'] = False

In [23]:
df['IsHazardousMaterial'].value_counts()

False    47
True      2
Name: IsHazardousMaterial, dtype: int64

In [24]:
df['expected_Ishazardous'].value_counts()

nan    39
no     10
Name: expected_Ishazardous, dtype: int64

I had not taken into account dg materials as hazardous in expected values, so making the change here based on predicted value. This is a temporary fix, ideally we should change the expected value in excel.

In [26]:
df.loc[df['IsHazardousMaterial']==True, 'expected_Ishazardous'] = 'yes'

**I haven't taken into account dg as hazardous in expected values, so I have to change it**

* Otherwise the above logic for predicted values are fine.

## Stackable vs Non Stackable

In [27]:
non_stack = ['non stackable', 'stackable no', 
                          'not stackable', 'non-stackable',
            'nonstackable','non - stackable',
             'not stackable','not-stackable','not - stackable',
            'non stack']
stack = ['stackable yes', 'stackable', 'stack']

def check_non_binary_in_row_with_space(strings_list, text_row):
    for string in strings_list:
        if(f" {string} " in text_row or f"{string} " in text_row or f" {string}" in text_row):
            return True
    return False

def check_binary_in_row_with_space(strings_list, text_row):
    for string in strings_list:
        if(f" {string} " in text_row or f"{string} " in text_row or f" {string}" in text_row):
            return True
    return False


df['IsStackable'] = np.nan

for i, row in df.iterrows():
    email = row['email_processed']
    if(check_non_binary_in_row_with_space(non_stack, email)):
        df.loc[i, 'IsStackable'] = False
    elif((df.loc[i, 'IsStackable'] != False)&(check_binary_in_row_with_space(stack, email))):
        df.loc[i, 'IsStackable'] = True
    else:
        df.loc[i, 'IsStackable'] = True

In [28]:
df['IsStackable'].value_counts()

True     41
False     8
Name: IsStackable, dtype: int64

In [29]:
df['expected_Isstackable'].value_counts()

nan    37
No     10
yes     2
Name: expected_Isstackable, dtype: int64

**I have assumed that a home theater is stackable, although no keyword is mentioned in the email**

* Apart from these corner cases the predicted logic is fine.

## Top Loadable vs Non Top Loadable

In [30]:
non_top = ['top loaded no', 'not toploadable']
top = ['top loadable', 'top load only', 
                        'top load']

df['IsToploaded'] = np.nan

for i, row in df.iterrows():
    email = row['email_processed']
    if(check_non_binary_in_row_with_space(non_top, email)):
        df.loc[i, 'IsToploaded'] = False
    elif((df.loc[i, 'IsToploaded'] != False)&(check_binary_in_row_with_space(top, email))):
        df.loc[i, 'IsToploaded'] = True
    else:
        df.loc[i, 'IsToploaded'] = True

In [31]:
df['IsToploaded'].value_counts()

True    49
Name: IsToploaded, dtype: int64

In [32]:
df['expected_Istoploaded'].value_counts()

nan    46
No      1
yes     1
Yes     1
Name: expected_Istoploaded, dtype: int64

In [33]:
df.loc[df['expected_Istoploaded']=='No']['email_processed'].values[0]

'rate request | vancouver, canada to highton, australia hi there, can we please get a rate for the following: please include door delivery, customs clearance, and dthc separately. from: vancouver, canada to: mark chapman 15 nelson ave highton vic 3216 australia 1 pallet 75x51x67 high (x1) commodity: commercial shipment - home theatre equipment'

**Again here in the expected values i have assumed that a home thetre is not toploadable**

In [34]:
df['expected_Istoploaded'].value_counts().index

Index(['nan', 'No', 'yes', 'Yes'], dtype='object')

Hardcoding this change in predicted value for home theatre, this we can discuss with business if they can provide a list of items and corresponding attributes of binary variable. Then we can confirm if a home theatre can be considered a toploadable item or not.

* There is only one entry corresponding to home theatre item, so changing it to False.

In [35]:
df.loc[df['email_processed'].str.contains('home theatre'), 'IsToploaded'] = False

In [36]:
df = df.rename(columns={"IsToploaded": 'IsTopLoaded'})

In [37]:
preds = list(['IsStackable', 'IsHazardousMaterial', 'IsTopLoaded'])
df_output = df.rename(columns={c: c+'_predicted' for c in preds})

In [38]:
df_output['expected_Isstackable'] = df_output['expected_Isstackable'].str.lower()
df_output['expected_Isstackable'] = df_output['expected_Isstackable'].fillna('yes')
df_output["expected_Isstackable"] = df_output["expected_Isstackable"].apply(lambda x: False if x.lower()=="no"  else True)

In [39]:
df = df_output.drop_duplicates(subset=['email_processed'], keep='first')
df = df.reset_index(drop=True)
df.shape

(49, 79)

In [40]:
# 'IsStackable_predicted', 'expected_Isstackable'
matched, un_matched = df[df['expected_Isstackable']==df['IsStackable_predicted']].shape[0],df[df['expected_Isstackable']!=df['IsStackable_predicted']].shape[0]
a = matched/(matched+un_matched)
print('{}% match of stackable'.format(round(a*100,2)))

95.92% match of stackable


### Accuracy of Binary Variables

In [41]:
# Auto encode the columns to be measured 

df['expected_Isstackable'] = np.where(df['expected_Isstackable']==True, 1, 0)
df['IsStackable_predicted'] = np.where(df['IsStackable_predicted']==True, 1, 0)
OriginType_Confusion_Matrix = pd.crosstab(df['IsStackable_predicted'], df['expected_Isstackable'])
OriginType_Confusion_Matrix.columns = ['Expected - nonstackable', 'Expected - stackable']
OriginType_Confusion_Matrix = OriginType_Confusion_Matrix.reset_index()
OriginType_Confusion_Matrix['IsStackable_predicted'] = np.where((OriginType_Confusion_Matrix['IsStackable_predicted'] == 0), 'Predicted - nonstackable', 'Predicted - stackable')
OriginType_Confusion_Matrix

,IsStackable_predicted,Expected - nonstackable,Expected - stackable
0,Predicted - nonstackable,8,0
1,Predicted - stackable,2,39


In [42]:
# Accuracy
OriginType_Accuracy = ((OriginType_Confusion_Matrix['Expected - nonstackable'][0] + OriginType_Confusion_Matrix['Expected - stackable'][1])/(OriginType_Confusion_Matrix['Expected - nonstackable'][0] + OriginType_Confusion_Matrix['Expected - stackable'][0] + 
OriginType_Confusion_Matrix['Expected - nonstackable'][1] + OriginType_Confusion_Matrix['Expected - stackable'][1]))*100
print('Accuracy of IsStackable : ' , OriginType_Accuracy)

# Recall of Door Class within OriginType
OriginType_DoorRecall = ((OriginType_Confusion_Matrix['Expected - nonstackable'][0])/(OriginType_Confusion_Matrix['Expected - nonstackable'][0] + 
OriginType_Confusion_Matrix['Expected - nonstackable'][1] ))*100
print('Recall of non stackable : ' , OriginType_DoorRecall)

# Precision of Door Class within OriginType
OriginType_DoorPrecision = ((OriginType_Confusion_Matrix['Expected - nonstackable'][0])/(OriginType_Confusion_Matrix['Expected - nonstackable'][0] + 
OriginType_Confusion_Matrix['Expected - stackable'][0] ))*100
print('Precision of non stackable class : ' , OriginType_DoorPrecision)

# Recall of Port Class within OriginType
OriginType_PortRecall = ((OriginType_Confusion_Matrix['Expected - stackable'][1])/(OriginType_Confusion_Matrix['Expected - stackable'][0] + 
OriginType_Confusion_Matrix['Expected - stackable'][1] ))*100
print('Recall of stackable class : ' , OriginType_PortRecall)

# Precision of Port Class within OriginType
OriginType_PortPrecision = ((OriginType_Confusion_Matrix['Expected - stackable'][1])/(OriginType_Confusion_Matrix['Expected - stackable'][1] + 
OriginType_Confusion_Matrix['Expected - nonstackable'][1] ))*100
print('Precision of stackable class : ' , OriginType_PortPrecision)

Accuracy of IsStackable :  95.91836734693877
Recall of non stackable :  80.0
Precision of non stackable class :  100.0
Recall of stackable class :  100.0
Precision of stackable class :  95.1219512195122


In [43]:
df['expected_Ishazardous'] = df['expected_Ishazardous'].astype(str).str.lower()
df['expected_Ishazardous'] = df['expected_Ishazardous'].fillna('no')
df["expected_Ishazardous"] = df["expected_Ishazardous"].apply(lambda x: True if "yes" in x.lower().strip()  else False)

In [44]:
# 'IsStackable_predicted', 'expected_Isstackable'
matched, un_matched = df[df['expected_Ishazardous']==df['IsHazardousMaterial_predicted']].shape[0],df[df['expected_Ishazardous']!=df['IsHazardousMaterial_predicted']].shape[0]
a = matched/(matched+un_matched)
print('{}% match of hazardous'.format(round(a*100,2)))

100.0% match of hazardous


In [45]:
# Auto encode the columns to be measured 

df['expected_Ishazardous'] = np.where(df['expected_Ishazardous']==True, 1, 0)
df['IsHazardousMaterial_predicted'] = np.where(df['IsHazardousMaterial_predicted']==True, 1, 0)
OriginType_Confusion_Matrix = pd.crosstab(df['IsHazardousMaterial_predicted'], df['expected_Ishazardous'])
OriginType_Confusion_Matrix.columns = ['Expected - nonhazardous', 'Expected - hazardous']
#OriginType_Confusion_Matrix['Expected - hazardous'] = 0
OriginType_Confusion_Matrix = OriginType_Confusion_Matrix.reset_index()
OriginType_Confusion_Matrix['IsHazardousMaterial_predicted'] = np.where((OriginType_Confusion_Matrix['IsHazardousMaterial_predicted'] == 0), 'Predicted - nonhazardous', 'Predicted - hazardous')
OriginType_Confusion_Matrix

,IsHazardousMaterial_predicted,Expected - nonhazardous,Expected - hazardous
0,Predicted - nonhazardous,47,0
1,Predicted - hazardous,0,2


In [46]:
# Accuracy
OriginType_Accuracy = ((OriginType_Confusion_Matrix['Expected - nonhazardous'][0] + OriginType_Confusion_Matrix['Expected - hazardous'][1])/(OriginType_Confusion_Matrix['Expected - nonhazardous'][0] + OriginType_Confusion_Matrix['Expected - hazardous'][0] + 
OriginType_Confusion_Matrix['Expected - nonhazardous'][1] + OriginType_Confusion_Matrix['Expected - hazardous'][1]))*100
print('Accuracy of IsHazardous : ' , OriginType_Accuracy)

# Recall of Door Class within OriginType
OriginType_DoorRecall = ((OriginType_Confusion_Matrix['Expected - nonhazardous'][0])/(OriginType_Confusion_Matrix['Expected - nonhazardous'][0] + 
OriginType_Confusion_Matrix['Expected - nonhazardous'][1] ))*100
print('Recall of nonhazardous : ' , OriginType_DoorRecall)

# Precision of Door Class within OriginType
OriginType_DoorPrecision = ((OriginType_Confusion_Matrix['Expected - nonhazardous'][0])/(OriginType_Confusion_Matrix['Expected - nonhazardous'][0] + 
OriginType_Confusion_Matrix['Expected - hazardous'][0] ))*100
print('Precision of non hazardous class : ' , OriginType_DoorPrecision)

# Recall of Port Class within OriginType
OriginType_PortRecall = ((OriginType_Confusion_Matrix['Expected - hazardous'][1])/(OriginType_Confusion_Matrix['Expected - hazardous'][0] + 
OriginType_Confusion_Matrix['Expected - hazardous'][1] ))*100
print('Recall of hazardous class : ' , OriginType_PortRecall)

# Precision of Port Class within OriginType
OriginType_PortPrecision = ((OriginType_Confusion_Matrix['Expected - hazardous'][1])/(OriginType_Confusion_Matrix['Expected - hazardous'][1] + 
OriginType_Confusion_Matrix['Expected - nonhazardous'][1] ))*100
print('Precision of hazardous class : ' , OriginType_PortPrecision)

Accuracy of IsHazardous :  100.0
Recall of nonhazardous :  100.0
Precision of non hazardous class :  100.0
Recall of hazardous class :  100.0
Precision of hazardous class :  100.0


In [47]:
df['expected_Istoploaded'] = df['expected_Istoploaded'].str.lower()
df.loc[df['expected_Istoploaded']=='nan', 'expected_Istoploaded'] = 'yes'
df["expected_Istoploaded"] = df["expected_Istoploaded"].apply(lambda x: True if "yes" in x.lower().strip() else False)

In [48]:
# 'expected_Istoploaded', 'IsTopLoaded_predicted'
matched, un_matched = df[df['expected_Istoploaded']==df['IsTopLoaded_predicted']].shape[0],df[df['expected_Istoploaded']!=df['IsTopLoaded_predicted']].shape[0]
a = matched/(matched+un_matched)
print('{}% match of toploadable'.format(round(a*100,2)))

100.0% match of toploadable


In [49]:
# Auto encode the columns to be measured 

df['expected_Istoploaded'] = np.where(df['expected_Istoploaded']==True, 1, 0)
df['IsTopLoaded_predicted'] = np.where(df['IsTopLoaded_predicted']==True, 1, 0)


OriginType_Confusion_Matrix = pd.crosstab(df['IsTopLoaded_predicted'], df['expected_Istoploaded'])
OriginType_Confusion_Matrix.columns = ['Expected - nontoploadable', 'Expected - toploadable']
OriginType_Confusion_Matrix = OriginType_Confusion_Matrix.reset_index()
OriginType_Confusion_Matrix['IsTopLoaded_predicted'] = np.where((OriginType_Confusion_Matrix['IsTopLoaded_predicted'] == 0), 'Predicted - nontoploadable', 'Predicted - toploadable')
# OriginType_Confusion_Matrix.loc[-1] = ['Predicted - nontoploadable',0, 0]  # adding a row
# OriginType_Confusion_Matrix.index = OriginType_Confusion_Matrix.index + 1  # shifting index
# OriginType_Confusion_Matrix = OriginType_Confusion_Matrix.sort_index()  # sorting by index
OriginType_Confusion_Matrix

,IsTopLoaded_predicted,Expected - nontoploadable,Expected - toploadable
0,Predicted - nontoploadable,1,0
1,Predicted - toploadable,0,48


In [50]:
# Accuracy
OriginType_Accuracy = ((OriginType_Confusion_Matrix['Expected - nontoploadable'][0] + OriginType_Confusion_Matrix['Expected - toploadable'][1])/(OriginType_Confusion_Matrix['Expected - nontoploadable'][0] + OriginType_Confusion_Matrix['Expected - toploadable'][0] + 
OriginType_Confusion_Matrix['Expected - nontoploadable'][1] + OriginType_Confusion_Matrix['Expected - toploadable'][1]))*100
print('Accuracy of istoploadable : ' , OriginType_Accuracy)

# Recall of Door Class within OriginType
OriginType_DoorRecall = ((OriginType_Confusion_Matrix['Expected - nontoploadable'][0])/(OriginType_Confusion_Matrix['Expected - nontoploadable'][0] + 
OriginType_Confusion_Matrix['Expected - nontoploadable'][1] ))*100
print('Recall of non toploadable : ' , OriginType_DoorRecall)

# Precision of Door Class within OriginType
OriginType_DoorPrecision = ((OriginType_Confusion_Matrix['Expected - nontoploadable'][0])/(OriginType_Confusion_Matrix['Expected - nontoploadable'][0] + 
OriginType_Confusion_Matrix['Expected - toploadable'][0] ))*100
print('Precision of non toploadable class : ' , OriginType_DoorPrecision)

# Recall of Port Class within OriginType
OriginType_PortRecall = ((OriginType_Confusion_Matrix['Expected - toploadable'][1])/(OriginType_Confusion_Matrix['Expected - toploadable'][0] + 
OriginType_Confusion_Matrix['Expected - toploadable'][1] ))*100
print('Recall of toploadable class : ' , OriginType_PortRecall)

# Precision of Port Class within OriginType
OriginType_PortPrecision = ((OriginType_Confusion_Matrix['Expected - toploadable'][1])/(OriginType_Confusion_Matrix['Expected - toploadable'][1] + 
OriginType_Confusion_Matrix['Expected - nontoploadable'][1] ))*100
print('Precision of toploadable class : ' , OriginType_PortPrecision)

Accuracy of istoploadable :  100.0
Recall of non toploadable :  100.0
Precision of non toploadable class :  100.0
Recall of toploadable class :  100.0
Precision of toploadable class :  100.0


<h1><center> Pattern Matching for the Remaining Non-Binary variables </h1></center>

In [51]:
# df[['email_processed']].values

In [53]:
def remove_letters(text):
    return ''.join([char for char in text if not char.isalpha()])

def remove_special_characters(text):
    pattern = r'[^a-zA-Z0-9\s]'
    clean_text = re.sub(pattern, '', text)
    return clean_text
# remove_special_characters, remove_non_digits
def remove_non_digits(text):
    clean_text = re.sub(r'[^0-9.]', '', text)
    return clean_text


def process_dimensions(text):
    # single quote implies inch, so replacing it to standard format
    text = text.replace("'", '"')
# r'\d+"\s*x\s*\d+"\s*x\s*\d+"\s*|"\s*x\s*\d+"\s*x\s*\d+"\s*
    regex_patterns = [
        r'\d+\s*x\s*(\d+x\d+x\d+)',
        r'(?<=\d\s)(\d+\s\d+\s\d+)',
        r'(\d+)"x(\d+x\d+)',
        r'"?(\d+(?:\.\d+)?)\s*x\s*(\d+(?:\.\d+)?)\s*x\s*(\d+(?:\.\d+)?)',
        r'(\d+(?:\.\d+)?)[\s]+(\d+(?:\.\d+)?)[\s]+(\d+(?:\.\d+)?)',
        r'"?(\d+(?:\.\d+)?)\s*x\s*(\d+(?:\.\d+)?)\s*x\s*(\d+(?:\.\d+)?)"?',
        r'(\d+(?:\.\d+)?)\s*x\s*(\d+(?:\.\d+)?)\s*x\s*(\d+(?:\.\d+)?)',
        r'(\d+(?:\.\d+)?)\s*x\s*(\d+(?:\.\d+)?)\s*x\s*(\d+(?:\.\d+)?)',
        r'(\d+(?:\.\d+)?)\s*\((\w+)\)\s*x\s*(\d+(?:\.\d+)?)\s*x\s*(\d+(?:\.\d+)?)',
        r'\d+(?:\s*x\s*\d+)',
        r'\d+\s*\*\s*\d+\s*\*\s*\d+\s*in',
        r'\d+"\s*x\s*\d+"\s*x\s*\d+\s*in',
        r'\d+\s*\*\s*\d+\s*\*\s*\d+',
        r'\d+\s*"\s*x\s*\d+"\s*x\s*\d+"\s*|"\s*x\s*\d+"\s*x\s*\d+"\s*',
        r'\d+"\s*x\s*\d+\s*x\s*\d+',
        r'(\d+)"x(\d+)x(\d+)',
        r'(\d+)\(h\)x(\d+)x(\d+)',
        r'(\d+(\.\d+)?)\s*x\s*(\d+(\.\d+)?)\s*x\s*(\d+(\.\d+)?)\s*cms',
        r'l:\s*(\d+\.\d+)\s*\|\s*w:\s*(\d+\.\d+)\s*\|\s*h:\s*(\d+\.\d+)\s*cm',
        r'(\d+)x(\d+)x(\d+)',
        r'height\s*:\s*([\d.]+)\s*inches\s*width\s*:\s*([\d.]+)\s*inches\s*length\s*:\s*([\d.]+)\s*inches',
        r'(\d+(?:\.\d+)?)\s*[/\s]*(\d+(?:\.\d+)?)\s*[/\s]*(\d+(?:\.\d+)?)\s*(\w+)',
        r'(\d+(?:\.\d+)?)\s*[\sxX]\s*(\d+(?:\.\d+)?)\s*[\sxX]\s*(\d+(?:\.\d+)?)\s*(\w+)',
        r'(\d+(?:\.\d+)?)\s*x\s*(\d+(?:\.\d+)?)\s*x\s*(\d+(?:\.\d+)?)']

    dims = []
    for pattern in regex_patterns:
        matches = re.findall(pattern, text)
        if matches:
            dims.extend(matches)
            break

    return dims

# df['email_processed'] = df['email_processed'].astype(str)
# df['email_processed'] = df['email_processed'].apply(preprocess_text)
# df['email_processed'] = df['email_processed'].str.replace('Review: ```',' ')
# df['email_processed'] = df['email_processed'].str.replace('Review:', ' ')
# df['email_processed'] = df['email_processed'].str.replace(' tml ', 'terminal')
# df['email_processed'] = df['email_processed'].str.replace('^\W+','' )
# df['email_processed'] = df['email_processed'].str.lower()


# change email_processed - run only on body
df['predicted_Dimensions_raw'] = df['email_processed'].apply(process_dimensions)

# Removing Square brackets - list

df['predicted_Dimensions'] = [','.join(map(str, l)) for l in df['predicted_Dimensions_raw']]


# # Function to determine unit based on value
# def assign_unit(value):
#     if "'" in value:
#         return "inch"
#     else:
#         return None


In [54]:
# Validation 

df[['emailid', 'email_processed','predicted_Dimensions_raw', 'predicted_Dimensions', 'expected_Dimensions']]#.head()

,emailid,email_processed,predicted_Dimensions_raw,predicted_Dimensions,expected_Dimensions
0,Simon Holt <Simon.holt@nordic-on.com>,vs: lcl quote request | from location : britis...,[],,nan
1,Linda-Nancy Colizza <Linda-Nancy.Colizza@hellm...,"santos good morning, need rate please, termina...","[(44, 44, 42)]","('44', '44', '42')",44 x44 x42
2,Sally Chieng <sallyc@itn-logistics.ca>,"rfq / itn door winnipeg, mb to cfs shanghai, c...","[(48, 48, 50)]","('48', '48', '50')",48x48 x50
3,Michelle Guo <michelle.guo@sparxlogistics.com>,re: quote from toronto terminal to london term...,"[48*40*70 in, 48*40*70 in, 48 *40*60 in, 48*40...","48*40*70 in,48*40*70 in,48 *40*60 in,48*40*76 in","48*40*70 in, 48*40*70 in, 48 *40*60 in, 48*40..."
4,HLIWA Bozena <Bozena.HLIWA@bollore.com>,rate request lcl cargo non stackable cfs termi...,"[(0.80, 1.20, 1.15)]","('0.80', '1.20', '1.15')",0.80x 1.20 x 1.15 h m
5,LCL Operations <lcl@goworldcargo.com>,"rate request | vancouver, canada to highton, a...","[(75, 51, 67)]","('75', '51', '67')",75x51x67
6,LCL Operations <lcl@goworldcargo.com>,"rate request | vancouver, canada to darlington...","[(122, 102, 161), (122, 102, 121)]","('122', '102', '161'),('122', '102', '121')","122 x 102 x 161 cm, 122 x 102 x 121 cm"
7,Jude Menezes <jude@zodiacimpex.com>,lcl rate request toronto to singapore you don'...,"[(89, 112, 132), (91.5, 91.5, 104)]","('89', '112', '132'),('91.5', '91.5', '104')","89 x 112 x 132 cms, 91.5 x 91.5 x 104 cms"
8,Sandro Avilez Caldart <savilez@willsonintl.com>,"lcl import | term toronto, on, ca to port of s...","[130"" x 90"" x 98 in, 80"" x 58"" x 84 in]","130"" x 90"" x 98 in,80"" x 58"" x 84 in","130"" x 90"" x 98 in, 80"" x 58"" x 84 in"
9,Sarah | 1UP Cargo Pricing <Sarah@1upcargo.com>,"fw: canada exw rates form door to nansha,guang...",[122*102*211],122*102*211,122*102*211cm


In [56]:
def process_quantity(text):
    text = text.replace("'", '')

    regex_patterns = [
        r'\d+ x pallets', r'\d+ x pallet',r'\d+ plt',r'\d+ plts',r'\d+ pallet', r'\d+ pallets',
        r'\d+xpallets', r'\d+xpallet',r'\d+plt',r'\d+plts',r'\d+pallet', r'\d+pallets',
         r'\d+ x skid', r'\d+ x skids', r'\d+ skid',r'\d+ skids' ,
        r'\d+xskid', r'\d+xskids', r'\d+skid',r'\d+skids' ,
        r'\d+ x crate', r'\d+ x crates', r'\d+ crate',r'\d+ crates',
        r'\d+ w/crate',r'\d+ w/crates', r'\d+ wooden crate',r'\d+ wooden crates',r'\d+ wc',
        r'\d+xcrate', r'\d+xcrates', r'\d+crate',r'\d+crates',
        r'\d+xpkg', r'\d+x pkg', r'\d+ pkgs',r'\d+ pkgs',r'\d+ pkg',
        r'\d+ pieces', r'\d+ total pieces',r'total pieces \d+',r'palets totals: \d+',
        
        r'\b\w+\s+wooden crate\b'
        ]

    qty = []
    for pattern in regex_patterns:
        matches = re.findall(pattern, text)
        if matches:
            qty.extend(matches)
            break

    return qty


# Function to calculate total quantity from comma-separated values
def calculate_total_quantity(value_str):
    if((',') in value_str):
        values = value_str.split(',')
        if(len(values)>1):
            quantity = sum(int(val.split()[0]) for val in values)
        else:
            quantity = value_str
    else:
        quantity = value_str
    return quantity

number_mapping = {
    "one": 1,
    "two": 2,
    "three": 3,
    "four": 4,
    "five": 5,
    "six": 6,
    "seven": 7,
    "eight": 8,
    "nine": 9,
    "ten": 10
    # Add more mappings as needed
}

def replace_alphabetical_numbers(text):
    words = text.lower().split(' ')  # Split the text into lowercase words
    replaced_words = []

    for word in words:
        if word in number_mapping:
            replaced_words.append(str(number_mapping[word]))
        else:
            replaced_words.append(word)

    replaced_text = ' '.join(replaced_words)
    return replaced_text


df['predicted_quantity_raw'] = df['email_processed'].apply(process_quantity)
# Removing Square brackets - list
df['predicted_quantity_raw'] = [','.join(map(str, l)) for l in df['predicted_quantity_raw']]
df['predicted_quantity_raw'] = df['predicted_quantity_raw'].apply(lambda text: replace_alphabetical_numbers(text))



# Fix scenarios where multiple digits are extracted
df['predicted_quantity_raw_1'] = df['predicted_quantity_raw'].apply(calculate_total_quantity)
df['predicted_quantity'] = df['predicted_quantity_raw_1']
df['predicted_quantity'] = df['predicted_quantity'].astype(str).apply(remove_non_digits)

In [57]:
# row 45 - addition of comma separated values, and "one wooden carret" include alphabetical combination
df[['emailid', 'email_processed','predicted_quantity_raw', 'predicted_quantity_raw_1', 'predicted_quantity','expected_Quantity']].sort_values(by = 'emailid')

,emailid,email_processed,predicted_quantity_raw,predicted_quantity_raw_1,predicted_quantity,expected_Quantity
34,"""Abbie Hood (UK-LON ECU Worldwide)"" <AbbieHood...",dap rate hello could i have dap charges for th...,4 x pallets,4 x pallets,4,4.0
30,"""Abbie Hood (UK-LON ECU Worldwide)"" <AbbieHood...",exw rate hello could i have exw charges for th...,1 plt,1 plt,1,1.0
13,"""Abbie Hood (UK-LON ECU Worldwide)"" <AbbieHood...",dap ratehello could i have dap charges for the...,total pieces 13,total pieces 13,13,13.0
36,"""Berner, Nadine / Kuehne + Nagel / Tor ZS-BA""\...","kn rate request/ vancouver to barraquilla, co/...",2 crate,2 crate,2,2.0
29,"""Bob Everett (UK-LON ECU Worldwide)"" <BobEvere...",fw: freight quote - jkf - canada - sea - ac go...,,,,1.0
45,"""Denislav Chipev (BG-SOF ECU Worldwide)""\n\t<D...","dap charges l4m 4y8, canada / 3 pcs hi, please...","1 pallet,1 pallet,1 pallet",3,3,3.0
12,"""Denislav Chipev (BG-SOF ECU Worldwide)""_x000D...","dap to v7w-1w1 west vancouver, british columbi...","1 pallet,1 pallet",2,2,1.0
39,"""JANI, TWISHA"" <twisha.jani@dbschenker.com>",re: new booking from montreal to shanghai hi a...,,,,nan
23,"""Karina Alonso (ES-BCN ECU Worldwide)"" <Karina...",hi all pls could you send us your better dap r...,palets totals: 2,palets totals: 2,2,2.0
18,"""Krizia Mejia"" <krizia.mejia@etheinsen.com.do>",rate request // lcl /canada - rd // raver rovr...,,,,1.0


### quanity indicator - mismatch

In [58]:
df['predicted_quantity'].value_counts().index# indicator - quan

Index(['', '2', '1', '4', '3', '5', '7', '13', '14'], dtype='object')

In [60]:
df['predicted_quantity'] = df['predicted_quantity'].astype(str)
df['predicted_quantity'] = df['predicted_quantity'].apply(remove_letters)
df['predicted_quantity'] = df['predicted_quantity'].apply(remove_non_digits)
df.loc[df['predicted_quantity']=='', 'predicted_quantity'] = np.nan
df['predicted_quantity'] = df['predicted_quantity'].fillna(0)
df['predicted_quantity'] = df['predicted_quantity'].apply(lambda x: float(x))
df.loc[df['predicted_quantity']>200, 'predicted_quantity'] = 1

In [61]:
df = df.rename(columns={'predicted_quantity': 'Quantity_predicted'})
df['Quantity_predicted'] = df['Quantity_predicted'].fillna(0)
df.loc[df['expected_Quantity']=='nan', 'expected_Quantity'] = 0
df['expected_Quantity'] = df['expected_Quantity'].astype(str).apply(remove_non_digits)
df['expected_Quantity'] = df['expected_Quantity'].astype(float, errors='ignore')
df['expected_Quantity'] = df['expected_Quantity'].replace('', 0)
df['expected_Quantity'] = df['expected_Quantity'].astype(float, errors='ignore')
#df['expected_Quantity'] = df['expected_Quantity'].fillna(0)
decimals = 1
df['expected_Quantity'] = df['expected_Quantity'].apply(lambda x: round(x, decimals))
df['Quantity_predicted'] = df['Quantity_predicted'].apply(lambda x: round(x, decimals))
# df['Quantity_predicted'] = df['Quantity_predicted'].astype(int)
# df['expected_Quantity'] = df['expected_Quantity'].astype(int)
matched, un_matched = df[df['expected_Quantity']==df['Quantity_predicted']].shape[0],df[df['expected_Quantity']!=df['Quantity_predicted']].shape[0]
a = matched/(matched+un_matched)
print('{}% match of quantity'.format(round(a*100,2)))

85.71% match of quantity


In [62]:
df.loc[df['expected_Quantity']!=df['Quantity_predicted']][['Quantity_predicted', 'expected_Quantity']]

,Quantity_predicted,expected_Quantity
6,2.0,0.0
12,2.0,1.0
15,0.0,1.0
18,0.0,1.0
29,0.0,1.0
31,0.0,1.0
44,1.0,17.0


I will ask chatgpt combinedly for quantity and weight at the end, if possible dimension also.

### 4. Weight 

In [63]:
def process_weight(text):
    text = text.replace("'", '')
    
    regex_patterns = [
    r'\d+(?:,\d{3})*(?:\.\d+)? kg ',
    r'\d+(?:,\d{3})*(?:\.\d+)? kgs ',
    r'\d+(?:,\d{3})*(?:\.\d+)? kg ',
    r'\d+(?:,\d{3})*(?:\.\d+)? kgs',
    r'(kg): \d+(?:,\d{3})*(?:\.\d+)?',
    r'kg \d+(?:,\d{3})*(?:\.\d+)?',
    r'kgs \d+(?:,\d{3})*(?:\.\d+)?',
        # have to include comma cases, have included it but its not working as expected
        # i am excluding other units except kg, to avoid false values. Its better to return none than to return anything
#     r'\d+(?:,\d{3})*(?:\.\d+)? lb',
#     r'\d+(?:,\d{3})*(?:\.\d+)? lbs',
#     r'\d+(?:,\d{3})*(?:\.\d+)?lb',
#     r'\d+(?:,\d{3})*(?:\.\d+)?lbs',
#     r'\d+(?:,\d{3})*(?:\.\d+)? pound',
#     r'\d+(?:,\d{3})*(?:\.\d+)? pounds',
#     r'\d+(?:,\d{3})*(?:\.\d+)?pound',
#     r'\d+(?:,\d{3})*(?:\.\d+)?pounds',
     ]


#     regex_patterns = [
#         r'\d+ kg ', r'\d+ kgs ', r'\d+ kg ',r'\d+ kgs',r'(kg): \d+',
#         r'kg \d+', r'kgs \d+',
#         r'\d+ lb', r'\d+ lbs', r'\d+lb',r'\d+lbs',
#         r'\d+ pound', r'\d+ pounds', r'\d+pound',r'\d+pounds',
#         ]


    qty = []
    for pattern in regex_patterns:
        matches = re.findall(pattern, text)
        if matches:
            qty.extend(matches)
            break

    return qty

df['predicted_Weight_raw'] = df['email_processed'].apply(process_weight)

df['predicted_Weight'] = [','.join(map(str, l)) for l in df['predicted_Weight_raw']]

# Fix scenarios where multiple digits are extracted

df['predicted_Weight'] = df['predicted_Weight'].str.extract('(\d+[.\d]*)')

In [64]:
# 13 - 65.00 - account for decimal
df[['emailid', 'email_processed','predicted_Weight_raw','predicted_Weight','expected_Weight']].sort_values(by = 'emailid')

,emailid,email_processed,predicted_Weight_raw,predicted_Weight,expected_Weight
34,"""Abbie Hood (UK-LON ECU Worldwide)"" <AbbieHood...",dap rate hello could i have dap charges for th...,[3400 kgs ],3400,3400 kg
30,"""Abbie Hood (UK-LON ECU Worldwide)"" <AbbieHood...",exw rate hello could i have exw charges for th...,[1044 kgs ],1044,1044 kg
13,"""Abbie Hood (UK-LON ECU Worldwide)"" <AbbieHood...",dap ratehello could i have dap charges for the...,"[6500.000 kgs , 6500.00 kgs ]",6500.000,6500 kg
36,"""Berner, Nadine / Kuehne + Nagel / Tor ZS-BA""\...","kn rate request/ vancouver to barraquilla, co/...",[],NaN,1632.9 kg
29,"""Bob Everett (UK-LON ECU Worldwide)"" <BobEvere...",fw: freight quote - jkf - canada - sea - ac go...,[],NaN,997 kg
45,"""Denislav Chipev (BG-SOF ECU Worldwide)""\n\t<D...","dap charges l4m 4y8, canada / 3 pcs hi, please...",[4604 kg ],4604,4604 kg
12,"""Denislav Chipev (BG-SOF ECU Worldwide)""_x000D...","dap to v7w-1w1 west vancouver, british columbi...",[150 kg ],150,150 kg
39,"""JANI, TWISHA"" <twisha.jani@dbschenker.com>",re: new booking from montreal to shanghai hi a...,[],NaN,nan
23,"""Karina Alonso (ES-BCN ECU Worldwide)"" <Karina...",hi all pls could you send us your better dap r...,[],NaN,4 kg
18,"""Krizia Mejia"" <krizia.mejia@etheinsen.com.do>",rate request // lcl /canada - rd // raver rovr...,[],NaN,2101.94 kg


In [65]:
df['predicted_Weight'] = df['predicted_Weight'].astype(float)
df.loc[df['predicted_Weight'].notnull(), 'predicted_Weight_unit']='kg'

In [66]:
df.loc[df['predicted_Weight'].isnull(), 'predicted_Weight']=''

In [67]:
df.loc[df['predicted_Weight']==''].shape

(26, 87)

## ChatGPT for weight and quantity

In [68]:
def extract_weight_from_quotations(email):
    prompt = f"""
    Extract these informations from the given quotation email.
    
    Weight - Compute total weight in kg if there are multiple packages. Consider gross weight over net weight.
    unit of weight - Compute the unit of weight.
    
    Weight: [Weight]
    Unit of Weight: [unit of weight]
    
    The response should include only 2 lines with 2 fields mentioned above in the same order. Return fields with "None" if there is insufficient information.
        
    Review: ```{email}```
    """
    
    response = get_completion(prompt)
#     weight = response.split(sep='\n')[0]
#     weight_unit = response.split(sep='\n')[1]
    
    return response

quotation_columns = ['weight', 'weight_unit']
#df[quotation_columns] = ['', '']

for i, value in enumerate(df['email_processed']):
    weight = df.loc[i, 'predicted_Weight']
    if((weight=='')):
        result = extract_weight_from_quotations(value)
        df.loc[i, 'predicted_Weight'] = result.split(sep='\n')[0]
        df.loc[i, 'predicted_Weight_unit'] = result.split(sep='\n')[1]

### Save intermediate pickle

In [69]:
# df.to_pickle('weight_extraction_24_8.pkl')

In [70]:
df = pd.read_pickle('weight_extraction_24_8.pkl')

In [71]:
df[['predicted_Weight', 'predicted_Weight_unit', 'expected_Weight']]

,predicted_Weight,predicted_Weight_unit,expected_Weight
0,Weight: None,Unit of Weight: None,nan
1,2146.0,kg,2146 kg
2,Weight: 8800 lbs,Unit of Weight: lbs,3991.61 kg
3,"Weight: 14,759.75 kgs",Unit of Weight: kgs,1859.75kgs
4,880.0,kg,880 kg
5,Weight: None,Unit of Weight: None,nan
6,1.0,kg,300 kg
7,1054.0,kg,1054 kg
8,3.0,kg,3315.76 kg
9,Weight: 4900kg,Unit of Weight: kg,4900 kg


In [72]:
def convert_pounds_to_kgs(weight_in_lbs):
    pounds_per_kg = 0.45359237
    weight_in_kgs = weight_in_lbs * pounds_per_kg
    return weight_in_kgs

df['predicted_Weight'] = df['predicted_Weight'].astype(str)
df['predicted_Weight'] = df['predicted_Weight'].str.replace('Weight: ','')
df['predicted_Weight'] = df['predicted_Weight'].str.replace('Weight:','')
df['predicted_Weight_unit'] = df['predicted_Weight_unit'].str.replace('Unit of Weight: ', '')
df['predicted_Weight_unit'] = df['predicted_Weight_unit'].str.replace('Unit of Weight:', '')
df['predicted_Weight'] = df['predicted_Weight'].fillna('')
df['predicted_Weight'] = df['predicted_Weight'].str.replace('None', '')
df.loc[df['predicted_Weight'].str.contains('lb|lbs|pounds'), 'predicted_Weight_unit'] = 'pounds'
df['predicted_Weight'] = df['predicted_Weight'].apply(remove_non_digits)
df.loc[df['predicted_Weight'] == '', 'predicted_Weight'] = np.nan
df.loc[df['predicted_Weight'].str.count('[.]')>1, 'predicted_Weight']= np.nan
df['predicted_Weight'] = df['predicted_Weight'].apply(float)
df.loc[df['predicted_Weight']>100000, 'predicted_Weight'] = 1
df.loc[df['predicted_Weight_unit'] == 'pounds', 'predicted_Weight'] = df['predicted_Weight'].apply(convert_pounds_to_kgs)

In [73]:
df['predicted_Weight'] = df['predicted_Weight'].astype(str)
df['predicted_Weight'] = df['predicted_Weight'].apply(remove_letters)
df['predicted_Weight'] = df['predicted_Weight'].apply(remove_non_digits)
df.loc[df['predicted_Weight']=='', 'predicted_Weight'] = np.nan
df['predicted_Weight'] = df['predicted_Weight'].fillna(0)
df['predicted_Weight'] = df['predicted_Weight'].apply(lambda x: float(x))

df = df.rename(columns={"predicted_Weight": 'weight_predicted'})
df['weight_predicted'] = df['weight_predicted'].fillna(0)
df.loc[df['expected_Weight']=='nan', 'expected_Weight'] = 0
df['expected_Weight'] = df['expected_Weight'].astype(str).apply(remove_non_digits)
df['expected_Weight'] = df['expected_Weight'].astype(float, errors='ignore')
df['expected_Weight'] = df['expected_Weight'].replace('', 0)
df['expected_Weight'] = df['expected_Weight'].astype(float, errors='ignore')
decimals = 1
df['expected_Weight'] = df['expected_Weight'].apply(lambda x: round(x, decimals))
df['weight_predicted'] = df['weight_predicted'].apply(lambda x: round(x, decimals))
matched, un_matched = df[df['expected_Weight']==df['weight_predicted']].shape[0],df[df['expected_Weight']!=df['weight_predicted']].shape[0]
a = matched/(matched+un_matched)
print('{}% match of weight'.format(round(a*100,2)))

77.55% match of weight


In [74]:
(41/49)

0.8367346938775511

In [79]:
df.loc[df['expected_Weight']!=df['weight_predicted']][['emailid','predicted_Weight_raw','weight_predicted', 'expected_Weight']]

,emailid,predicted_Weight_raw,weight_predicted,expected_Weight
3,Michelle Guo <michelle.guo@sparxlogistics.com>,[],14759.8,1859.8
6,LCL Operations <lcl@goworldcargo.com>,[kg 1],1.0,300.0
8,Sandro Avilez Caldart <savilez@willsonintl.com>,"[3,316 kgs , 1,434 kgs ]",3.0,3315.8
14,"""Muna Khalifa (DHL CA)"" <muna.khalifa@dhl.com>",[],57.0,25.9
15,0,[],69.9,69.8
21,Maria Aragao <Maria.Aragao@craneww.com>,[],1449.7,1404.3
27,Gelena | Sealion Cargo <gelena@sealioncargo.com>,[],1500.0,680.4
29,"""Bob Everett (UK-LON ECU Worldwide)"" <BobEvere...",[],0.0,997.0
37,Mike Gray <info@mglogistics.ca>,"[1,212.0 kg ]",1.0,1212.0
38,JALI Ussama <ussama.jali@bollore.com>,[],3155.0,1657.9
